# micro diffusion

https://huggingface.co/datasets/JourneyDB/JourneyDB

## 概要

micro diffusionは、低コストで学習可能なテキストから画像を生成する（T2I）拡散Transformerモデル（DiT）

- T2I = Text to Image
- DiT = Diffusion Transformer
- Vision Transformer（ViT）ベースの潜在拡散モデル（Latent Diffusion Models）

特徴:

- 訓練時に画像のパッチを最大75%をランダムにマスキングし、Transformerの計算量を削減
- マスキングの直前にPatch-mixerという前処理を挟む遅延マスキング（Differed Masking）を採用し、生成品質を改善
- MoE（Mixture-of-Experts）など最新の手法を採用し、生成品質を改善（合計11.6億パラメータ）
- 訓練データに合成データ（Synthetic images）を使用し、生成品質を改善
- 公開されている3700万枚の実画像と、追加の合成画像を訓練に使用
- 訓練に要したコストは1,890ドル（30万円）で、Stable Diffusionなどと匹敵する性能（FIDスコア）
    - 8台のH100 GPUを単一ノードで2.6日
    - Stable Diffusionの訓練コスト（3,345万円）より118倍低い
    - 最先端の手法（430万円）より14倍低い

micro_diffusionの生成例（512x512）:

![](image/fig1a.png)

左: Image of an atronaut riding a hourse in ___ styleの比較, 右: FIDスコアの比較:

![](image/fig1b.png)

- origami
- pixel art
- line art
- cyberpunk
- Van Goph Strarry Night

## 拡散モデルの前提知識

データ分布$\mathcal{D}$から、画像$x$とキャプション$c$をサンプルするとき、次式で表現する:

$$
(x, c) \sim \mathcal{D}
$$

画像$x$に、分散$\sigma^2$のガウスノイズを加えたとき、画像分布は次式で表現できる:

$$
p(x; \sigma)
$$

![](image/gaussian.png)

拡散ベースの確率モデルは、順拡散過程でノイズを加えた画像を、逆拡散過程で復元することを学習する:

![](image/diffusion_process.png)

- ノイズを加える方向が順拡散過程（Forward Diffusion Process）
- ノイズを除去する方向が逆拡散過程（Reverse Denoising Process, Reverse Diffusion Process）
- ノイズを加算/減算するタイムステップ$t$では、時間依存のガウスノイズ$\sigma(t)$を使用する
- 生成プロセスは、$p(x;\sigma_{max})$からノイズをサンプリングし、反復的にノイズ除去する仕組み

拡散過程を「決定論的で連続的なステップ」とすると次の常微分方程式で表現できる（ODE定式化）:

$$
dx = -\dot{\sigma}(t)\sigma(t)\nabla_{x}\log p(x;\sigma(t))dt
$$

- 決定論的（derteministic sampling）: ノイズ除去にランダム性が無い制約条件
- 常微分方程式（ODE, Ordinary Differential Equation）: 1つの独立変数に対する関数の変化を記述する式
- $dx$: 画像$x$の微小変化量
- $\sigma(t)$: タイムステップ$t$でのノイズ
- $\dot{\sigma}(t)$: タイムステップ$t$でのノイズの変化量（ドットは微分）
- $\nabla_{x}\log p(x;\sigma(t))$: より画像らしくなる方向の傾き（確率密度関数の勾配、スコア関数）
- $dt$: タイムステップの微小変化量
- タイムステップが正方向（$0$から$t$）のとき順拡散過程で、負の方向のとき逆拡散過程
- 推論時の実装に使用（最短ステップで画像生成できる）

拡散過程を「確率的で連続的なステップ」とすると次の確率微分方程式で表現できる（SDE定式化）:

$$
dx = [-\dot{\sigma}(t)\sigma(t)\nabla_{x}\log p(x;\sigma(t)) - \beta(t)\sigma(t)^{2}\nabla_{x}\log p(x;\sigma(t))]dt + \sqrt{2\beta(t)}\sigma(t)dw_{t}
$$

- $[-\dot{\sigma}(t)\sigma(t)\nabla_{x}\log p(x;\sigma(t)) - \beta(t)\sigma(t)^{2}\nabla_{x}\log p(x;\sigma(t))]dt$
    - 決定論的な拡散過程（Drift項）
    - $- \beta(t)\sigma(t)^{2}\nabla_{x}\log p(x;\sigma(t))$は、安定化のための値
- $\sqrt{2\beta(t)}\sigma(t)dw_{t}$
    - ランダムなノイズ（Diffusion項）
- $\beta(t)$: ノイズ強度を示すパラメータ
- $dw_t$: ランダムな微小ノイズ（標準ウィーナー過程）
- 学習時の実装に使用（ランダム性がモデルを強くする）

数学的にスコア関数は「ノイズ除去器が予測する修正方向」とほぼ同じ:

$$
\nabla_x \log p(x; \sigma(t)) \approx (F_\theta(x, \sigma(t)) - x) / \sigma(t)^2
$$

- $F_\theta(x, \sigma(t))$
    - ノイズ除去関数
    - 通常、ディープニューラルネットワークでモデル化される

T2Iの場合、ノイズ除去器はノイズ分布とキャプションを受け取る:

$$
F_\theta(x + \epsilon; \sigma(t), c)
$$

- $x + \epsilon$: 元画像$x$にノイズ$\epsilon$を足したノイズあり画像
- $c$: キャプション

T2Iの損失関数は次式で表現できる:

$$
\mathcal{L} = \mathbb{E}_{(x,c)\sim\mathcal{D}}\mathbb{E}_{\epsilon\sim\mathcal{N}(0,\sigma(t)^{2}I)}||F_{\theta}(x+\epsilon;\sigma(t),c)-x||_{2}^{2}
$$

- $\mathbb{E}_{(x,c)\sim\mathcal{D}}$: データ分布$D$から画像$x$とキャプション$c$をサンプルしたときのの損失の平均
- $\mathbb{E}_{\epsilon\sim{\mathcal{N}(0,\sigma(t)^2I)}}$: 分散$\sigma(t)^2$のガウスノイズからノイズ$\epsilon$をサンプルしたときの損失の平均
- $||...||_2^2$: 二乗誤差
- $F_\theta(\cdot)$: パラメータ$\theta$のノイズ除去器
- $x$: 正解の元画像
- ノイズ除去器が正しく元の画像を予測するように訓練する

訓練中のノイズの強度は、対数正規分布（log-normal distribution）を使用する:

$$
\ln(\sigma) \sim \mathcal{N}(P_{mean}, P_{std})
$$

![](image/lognormal.png)

- 画像の解像度が高くなるほど、信号対雑音比（SNR）が増加するため、分布を右にシフトする（平均と標準偏差を変える）
    - SNRは、高解像度になるほど（画像が大きくなるほど）ノイズが相対的に小さくなるので、増加する
    - SNRが高いと、ノイズの影響が少なくなり学習の効率が悪くなるため、右にシフトしてノイズを増やす

生成画像に対するプロンプトの強度を制御するために、CFG（Classifier-free Guidance）を採用:

$$
F_{\theta}(x;\sigma(t),c) = F_{\theta}(x;\sigma(t)) + w \cdot (F_{\theta}(x;\sigma(t),c) - F_{\theta}(x;\sigma(t)))
$$

- $F_\theta(x; \sigma(t), c)$: キャプションありの予測（無条件の予測）
- $F_\theta(x; \sigma(t))$: キャプションなしの予測（条件付きの予測）
- $w$: プロンプトの強度（Guidance Scale）（$w \ge 1$）。
- 条件付きの予測 = 無条件の予測 + 強度 * （条件付きの予測 - 無条件の予測）
- プロンプト無しでの生成に対応するため、学習中はキャプションの一部をランダムにドロップ

潜在拡散モデル（Latent Diffusion Models, LDM）は、ピクセル空間の画像ではなく、圧縮した潜在表現を学習に使用する:

- ピクセル空間: $\mathbb{R}^{h\times w\times 3}$
- 潜在空間（latent space）: $\mathbb{R}^{\frac{h}{n}\times\frac{w}{n}\times c}$
    - $n$: 圧縮率
    - $c$: 潜在空間のチャンネル数
- 画像から潜在表現（encoding）、潜在表現から画像（reverse mapping）では、VAE（Variable Auto Encoder）を使用
- LDMにより、学習の収束を早めることが可能

拡散Transformer（Diffision Transformer, DiT）モデル$F_\theta$は、$k$個のTransformerブロックで構成:

- 各Transformerブロックの構成
    - マルチヘッドセルフアテンション（Self-attention）
    - マルチヘッドクロスアテンション（Cross-attention）
    - フィードフォワードネットワーク（Feed-forward layer）
- 全ての画像は、パッチのシーケンスに変換される（ViTと同様）
- 全ての線形層の次元数を$d$とする

## DiTの計算コスト削減

DiTの計算コストを削減するために様々なアプローチがある:

![](image/fig2.png)

- a: 削減しない場合、$\text{サンプル数}M\times \text{モデルのパラメータ数}N \times \text{パッチ数}S$に比例する計算コストがかかる
- b: パッチを大きくする場合、VAEの圧縮率が高くなるため性能が低下する可能性がある
- c: パッチをドロップする手法はよく使われるが、75%捨てると性能が低下する
- d: パッチをドロップし、ドロップされた箇所に学習可能なパッチを差し込み、それをデコーダーが復元する（MaskDiT）
- e: パッチに軽量なTransformerを適用し（Patch-mixer）、ドロップする手法（Deferred masking）

Deferred maskingでは、Patch-mixerにより、マスクする前にパッチに画像全体の情報を埋め込む:

- Patch-mixerは、わずかな層で構成される軽量なTransformer
- 75%ドロップした場合でも性能を維持できる

Deferred maskingの損失関数は次式で表される:

$$
\mathcal{L} = \mathbb{E}_{(x,c)\sim\mathcal{D}}\mathbb{E}_{\epsilon\sim\mathcal{N}(0,\sigma(t)^{2}I)} || F_{\theta}(M_{\phi}(x+\epsilon) \odot (1-m); \sigma(t), c) - x \odot (1-m) ||_{2}^{2}
$$

- $M_\phi(x + \epsilon)$: パッチミキサーを適用し、画像全体の情報を混ぜ合わせる
- $\cdot (1 - m)$: パッチミキサーの出力にバイナリマスク$m$を適用する
- あとはDiT本来の損失関数と同じ形式
- 実際の訓練では、マスクありの事前学習に、マスキングなしのファインチューニングを行う

micro diffusionでは、MoE（Mixture-of-Experts）を採用:

- 学習コストを大幅に増やすことなく、パラメータ数とモデルの表現力を向上
- Expert-choice-routingにより、負荷分散のための補助損失関数が不要な設計

Transformerブロックを層ごとにスケーリングする手法も検討（Layer-wise scaling）:

- ネットワークの深い層は、浅い層よりも多くのパラメータが割り当てられる
- 視覚モデルの深い層はより複雑な特徴を学習する傾向があるため

micro diffusionのDiTの全体的なアーキテクチャ:

![](image/fig3.png)

## 実験

アーキテクチャ:

- [2つのバージョンのDiTアーキテクチャ][3]を使用:
    - DiT-Tiny/2: 小規模な実験用
    - DiT-XL/2: 大規模な実験用
- パッチサイズは2
    - 潜在空間の$2\times 2$グリッド
    - ピクセル空間だと$16\times 16$
- Transformerブロックを一つおきに、8つのエキスパートをもつMoEブロックに置き換える
- 画像の潜在表現の抽出に、[Stable Diffusion XL][2]で使用されている4チャネルのVAEを使用
    - [最新の16チャネルVAE][1]でも実験
- テキストエンコーダーは[CLIP][4]の一部を使用
    - [T5-xxl][5]は性能が高いが、計算コストとストレージコストが高いため今回は使用しない
- Patch-Mixerは、4つのTransformerブロックで構成（バックボーンの10%未満のパラメータ数）

[1]: https://arxiv.org/abs/2403.03206
[2]: https://arxiv.org/abs/2307.01952
[3]: https://arxiv.org/abs/2212.09748
[4]: https://arxiv.org/abs/2212.07143
[5]: https://arxiv.org/abs/1910.10683

訓練:

- AdamW最適化関数を使用
- コサイン学習率減衰（Cosine learnig rate decay）を使用
- 高い重み減衰（Weight decay）を使用
- 拡散モデルの基本設定は[EDMフレームワーク][1]を使用

[1]: https://arxiv.org/abs/2206.00364

学習データセット:

3つの実画像データセットを使用（合計2200万ペアの画像とテキスト）:

- [Conceptual Captions（CC12M）][1]
- [Segment Anything（SA1B）][2]
    - キャプションはLLaVAで作成
- [TextCaps][3]

大規模学習では更に2つの合成画像データセットを追加（合計1500万ペア）:

- [JourneyDB][4]
- [DiffusionDB][5]

小規模の比較実験用にデータセットを作成:

- cifar-captions
    - [COYO-700M][6]からCIFAR-10の10クラスの画像をサンプリングしたもの

[1]: https://arxiv.org/abs/2102.08981
[2]: https://arxiv.org/abs/2304.02643
[3]: https://arxiv.org/abs/2003.12462
[4]: https://arxiv.org/abs/2307.00716
[5]: https://arxiv.org/abs/2210.14896
[6]: https://github.com/kakaobrain/coyo-dataset

評価指標:

- [FID（Fréchet Inception Distance）][1]
    - 生成画像の品質と多様性を測る指標
    - Inception-v3モデルを使用したものと、CLIpモデルを使用したもので測定
    - 低いほどよい
- [CLIPスコア][2]
    - 画像とテキストの整合性を測る指標
    - 高いほどよい

[1]: https://arxiv.org/abs/1706.08500
[2]: https://arxiv.org/abs/2104.08718

## Deferred Maskingの有効性評価

評価設定:

- DiT-Tiny/2モデルを使用
- cifar-captionsデータセット（256x256）を使用
- 各モデルを60,000ステップで学習
- AdamW最適化関数を使用
- 最後の10,000ステップで平滑化係数0.995のEMA（Exponential Moving Average）を有効化

Deferred Maskingは、マスクが75%を超えると優れた性能を示す:

![](image/fig4.png)

デフォルト（Out-of-box）からハイパーパラメータを改良していった経過:

![](image/fig5a.png)

- Out-of-box: 改良前
    - マスキング率 75%
    - 学習率 $1.6\times 10^{-4}$
    - 重み減衰 0.01
    - ノイズ分布 EDM推奨の$(P_\text{mean}, P_\text{std}) = (-1.2, 1.2)$
- Higher weight decay: 強い重み減衰
    - 0.01から0.1へ
- Sigma distribution: ノイズ分布の変更
    - $(-0.6, 1.2)$に変更
    - 画質（FID）は下がったが、指示追従性（Clip-score）を高めた
- Larget patch-mixer: パッチミキサーのパラメータ数を増やす
    - FIDが大幅に改善
- Higher learning rate: 学習率を高めて、訓練時間を短縮
    - 学習ステップ数が少ないため、学習が不安定になる手前の最大値まで学習率を上げるとFIDも改善する

ランダムマスキングとブロックマスキングの比較実験では、前者のほうが性能が高かった:

![](image/fig5b.png)

## アーキテクチャ改良の検証

Transformerブロックを層ごとにスケーリングすると性能が改善（Layer-wise scaling）:

![](image/table1.png)

MoEブロックの使用は改善が見られなかった:

- CLIPスコアが、28.11から28.66へわずかに改善
- FIDスコアが、6.92から6.98へ悪化
- 固定の訓練ステップ数でMoEを使用すると、エキスパートの学習量が相対的に減るためだと仮説

Patch-mixerを使用すると、高いマスク率でMaskDiTよりも性能が向上:

![](image/fig6.png)

## Deferred Maskingの有効性の比較

総学習計算量をかえずに、TransformerのダウンスケーリングとDeferred Maskingを比較:

![](image/fig7a.png)

同一計算コストで、マスク無しの事前学習（破線）とマスクありの事前学習＋マスクなしファインチューニング（青線）を比較:

![](image/fig7b.png)

大規模モデルの2段階学習における計算コスト:

![](image/table2.png)

1. $256\times 256$の解像度でマスクありで学習
2. $256\times 256$の解像度でマスクなしで学習
3. $512\times 512$の解像度でマスクありで学習
4. $512\times 512$の解像度でマスクなしで学習

## 画像生成能力の検証

実画像と合成画像を組み合わせた3700万枚の画像を使用

また、40%の合成画像を含む画像のデータセットで比較実験

8つのエキスパートを持つTransformerブロックを1つおきに配置したDiT-XL/2を使用（MicroDiT）

学習プロセス:

- フェイズ1
    - $256\times 256$の画像を使用
    - 75％のバッチマスキングで25万ステップし、バッチマスキング無しで3万ステップのファインチューニング
- フェイズ2
    - $512\times 512$の画像を使用
    - 75%のバッチマスキングで5万ステップファインチューニングし、バッチマスキング無しで5000ステップを最適化

実画像のみと、実画像＋合成画像でのGPT-4oによる評価結果:

![](image/fig8.png)

実画像のみと、実画像＋合成画像で訓練したモデルの比較:

![](image/fig9.png)

## 潜在空間のチャンネル数の検証

限られた計算コストの場合、16チャネルよりも4チャネルのオートエンコーダを使用したほうが性能が高い:

![](image/fig10.png)

## 先行研究との比較

COCOデータセットにおけるゼロショット画像生成の比較:

![](image/table3.png)

GenEvalによる細かい画像生成タスクでの比較:

![](image/table4.png)

## 実装

### 環境構築

In [ ]:
import logging
import os
import platform
import subprocess

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)

PYTHON_VERSION = platform.python_version()
logger.info(f"Python {PYTHON_VERSION}")

NVIDIA_SMI = subprocess.run("nvidia-smi", capture_output=True, text=True).stdout
logger.info(f"{NVIDIA_SMI}")

In [ ]:
%pip install \
    "mosaicml[tensorboard, wandb]" \
    accelerate \
    beautifulsoup4 \
    datasets \
    diffusers \
    easydict \
    einops \
    fastparquet \
    huggingface_hub \
    hydra-core \
    mosaicml-streaming \
    omegaconf \
    open_clip_torch \
    pandas \
    timm \
    torchmetrics \
    tqdm \
    transformers

In [ ]:
from accelerate import Accelerator
from collections.abc import Iterable
from composer import Trainer
from composer.algorithms import GradientClipping
from composer.algorithms.low_precision_layernorm import apply_low_precision_layernorm
from composer.callbacks import LRMonitor
from composer.callbacks import OptimizerMonitor
from composer.callbacks import RuntimeEstimator
from composer.callbacks import SpeedMonitor
from composer.core import Precision
from composer.loggers import TensorboardLogger
from composer.loggers.wandb_logger import WandBLogger
from composer.models import ComposerModel
from composer.optim import CosineAnnealingWithWarmupScheduler
from composer.optim import ConstantScheduler
from composer.utils import dist, reproducibility
from diffusers import AutoencoderKL
from diffusers import AutoencoderKL
from diffusers.models.modeling_outputs import AutoencoderKLOutput
from easydict import EasyDict
from einops import rearrange
from functools import partial
from glob import glob
from huggingface_hub import hf_hub_download
from hydra import compose, initialize_config_dir
from itertools import repeat
from micro_diffusion.models.callbacks import LogDiffusionImages
from micro_diffusion.models.callbacks import NaNCatcher
from micro_diffusion.models.utils import text_encoder_embedding_format
from multiprocessing import Pool
from omegaconf import DictConfig, OmegaConf
from PIL import Image, UnidentifiedImageError
from streaming import MDSWriter
from streaming import Stream, StreamingDataset
from streaming.base import MDSWriter
from streaming.base.util import merge_index
from timm.models.vision_transformer import PatchEmbed
from torch.utils.data import DataLoader
from torchmetrics import Metric
from torchvision import transforms
from tqdm import tqdm
from transformers import CLIPTextModel, CLIPTokenizer, T5EncoderModel, T5Tokenizer
from typing import Callable, Dict, List, Optional, Sequence, Union
from typing import List, Dict, Union, Optional
from typing import List, Optional
from typing import Optional, Tuple, Dict, Union, List, Any
import hydra
import json
import math
import numpy as np
import open_clip
import os
import shutil
import subprocess
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers

# cuDNN(CUDA Deep Neural Network library）による最適化を有効化
# 3-5%の速度向上が見込まれる
torch.backends.cudnn.benchmark = True

# 乱数シードの設定
seed = 18
reproducibility.seed_all(seed)

logger.info(f"PyTorch {torch.__version__}")
logger.info(f"Transformers {transformers.__version__}")

In [ ]:
USER_ROOT = os.path.expanduser("~")
CACHE_DIR = os.path.join(USER_ROOT, ".cache", "micro_diffusion")

DATA_DIR = os.path.join(CACHE_DIR, "data")
os.makedirs(DATA_DIR, exist_ok=True)
logger.info(f"{DATA_DIR=}")

MODEL_DIR = os.path.join(CACHE_DIR, "models")
os.makedirs(MODEL_DIR, exist_ok=True)
logger.info(f"{MODEL_DIR=}")

In [ ]:
DATA_TYPES = {
    'float16': torch.float16,
    'bfloat16': torch.bfloat16,
    'float32': torch.float32
}

## DiT

![](image/dit.png)

### ヘルパー関数

In [ ]:
def modulate(x: torch.Tensor, shift: torch.Tensor, scale: torch.Tensor) -> torch.Tensor:
    """
    入力テンソルの平均をずらし（shift）、スケールを調整する
    Adaptive Layer Normalization（AdaLN）の手法の一部として使用される
    条件付けを特徴表現に適用するために使用
    shiftとscaleは、タイムステップtとテキスト埋め込みcをMLPに入力して予測する

    y = x * (1 + scale) + shift

    Args:
        x (torch.Tensor): 入力テンソル
        shift (torch.Tensor): シフト量
        scale (torch.Tensor): スケール量
    Returns:
        torch.Tensor: 変調後のテンソル
    """
    return x * (1 + scale.unsqueeze(1)) + shift.unsqueeze(1)

In [ ]:
def create_norm(norm_type: str, dim: int, eps: float = 1e-6) -> nn.Module:
    """
    レイヤー正規化層を作成する
    https://docs.pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html

    Args:
        norm_type (str): 正規化の種類
            - layernorm:
                - バイアスを持たないLayerNorm
            - np_layernorm:
                - 学習可能なパラメータを持たないLayerNorm
                - AdaLNの変調はmodulate関数で行うため
        dim (int): 正規化する次元のサイズ
        eps (float): 数値安定性のために使用される小さな値
    Returns:
        nn.Module: 指定された正規化層
    """
    if norm_type == "layernorm":
        return nn.LayerNorm(dim, eps=eps, bias=False)
    elif norm_type == "np_layernorm":
        return nn.LayerNorm(dim, eps=eps, elementwise_affine=False, bias=False)
    else:
        raise ValueError('Norm type not supported!')

### SelfAttention

In [ ]:
class SelfAttention(nn.Module):
    """
    DiT内部のマルチヘッド・セルフ・アテンション層

    - QK-Normで学習を安定化
    - FlashAttentionで計算を高速化・VRAM使用量削減
    - スケール・バイアスなしのレイヤー正規化層を使用
    """

    def __init__(
        self,
        dim: int,
        num_heads: int,
        qkv_bias: bool = True,
        norm_eps: float = 1e-6,
        hidden_dim: Optional[int] = None
    ):
        """
        Args:
            dim: Transformerの入力および出力テンソルの次元
            num_heads: アテンションヘッドの数
            qkv_bias: QKV線形層でバイアスを使用するかどうか
            norm_eps: 正規化層のゼロ除算防止用イプシロン
            hidden_dim: QKV空間の次元。Noneの場合、入力次元と同じ
        """
        logger.info(f"マルチヘッドセルフアテンションを初期化 {dim=} {num_heads=} {qkv_bias=} {norm_eps=} {hidden_dim=}")
        super().__init__()

        # 1) 属性の初期化

        self.dim = dim

        if hidden_dim is None:
            hidden_dim = dim

        assert hidden_dim % num_heads == 0, 'dim should be divisible by num_heads'

        self.hidden_dim = hidden_dim

        self.num_heads = num_heads

        self.head_dim = hidden_dim // num_heads
        logger.debug(f"{self.head_dim=}")

        self.qkv_bias = qkv_bias

        # 2) モジュールの初期化

        # QKVを同時に計算する線形層
        self.qkv = nn.Linear(dim, hidden_dim * 3, bias=qkv_bias)

        # 出力の線形層
        self.proj = nn.Linear(hidden_dim, dim, bias=qkv_bias)

        # Qの正規化層
        self.ln_q = create_norm(
            'np_layernorm', dim=hidden_dim, eps=norm_eps
        )

        # Kの正規化層
        self.ln_k = create_norm(
            'np_layernorm', dim=hidden_dim, eps=norm_eps
        )

    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        """
        マルチヘッド・セルフ・アテンションの順伝播

        Args:
            x (torch.Tensor): 入力テンソル
            mask (Optional[torch.Tensor]): マスクテンソル（未使用）
        Returns:
            torch.Tensor: 出力テンソル
        """
        logger.debug(f"マルチヘッドセルフアテンションの順伝播 {x.shape=}")

        # B: バッチサイズ, N: シーケンス長, C: チャンネル数
        B, N, C = x.shape

        # 1) QKVの計算

        # (B, N, C) -> (B, N, 3 * hidden_dim)
        # -> (B, N, 3, num_heads, head_dim)
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim)

        # Q, K, Vに分割
        # unbindはテンソルを指定した次元で分割し、タプルとして返す
        # それぞれ (B, N, num_heads, head_dim)
        q, k, v = qkv.unbind(2)

        # 2) QK-Normの適用

        # (B, N, num_heads, head_dim) -> (B, N, num_heads * head_dim)
        # -> レイヤー正規化 -> (B, N, num_heads, head_dim)
        q = self.ln_q(
            q.view(B, N, self.num_heads * self.head_dim)
        ).view(B, N, self.num_heads, self.head_dim).to(q.dtype)

        # 同様にKにも適用
        k = self.ln_k(
            k.view(B, N, self.num_heads * self.head_dim)
        ).view(B, N, self.num_heads, self.head_dim).to(k.dtype)

        # 3) スケールド・ドットプロダクト・アテンションの計算（FlashAttention使用）

        # (B, N, num_heads, head_dim) -> (B, num_heads, N, head_dim)
        # -> FlashAttention -> (B, num_heads, N, head_dim)
        # -> (B, N, num_heads, head_dim)
        x = torch.nn.functional.scaled_dot_product_attention(
            q.transpose(1, 2),
            k.transpose(1, 2),
            v.transpose(1, 2),
            is_causal=False
        ).transpose(1, 2).contiguous()
        
        # 4) 出力層の適用

        # ヘッドを結合
        # (B, N, num_heads, head_dim) -> (B, N, hidden_dim)
        x = x.view(B, N, self.num_heads * self.head_dim)

        # 線形層を適用
        x = self.proj(x)

        return x
    
    def custom_init(self, init_std: float) -> None:
        """
        重みの初期化をカスタマイズ
        """
        logger.info(f"SelfAttentionの重みをカスタム初期化 {init_std=}")
        nn.init.trunc_normal_(self.qkv.weight, mean=0.0, std=0.02)
        nn.init.trunc_normal_(self.proj.weight, mean=0.0, std=init_std)

### CrossAttention

In [ ]:
class CrossAttention(nn.Module):
    """
    マルチヘッド・クロスアテンション層
    DiTのテキスト条件付けに使用される

    - クエリは画像特徴量から計算し、キーとバリューはテキスト埋め込みから計算される
    - QK-Normで学習を安定化
    - FlashAttentionで計算を高速化・VRAM使用量削減
    """
    def __init__(
        self,
        dim: int,
        num_heads: int,
        qkv_bias: bool = True,
        norm_eps: float = 1e-6,
        hidden_dim: Optional[int] = None
    ):
        """
        Args:
            dim: 入力および出力テンソルの次元
            num_heads: アテンションヘッドの数
            qkv_bias: QKV線形層でバイアスを使用するかどうか
            norm_eps: 正規化層のゼロ除算防止用イプシロン
            hidden_dim: QKV空間の次元（Noneの場合、入力次元と同じ）
        """
        logger.info(f"マルチヘッドクロスアテンションを初期化 {dim=} {num_heads=} {qkv_bias=} {norm_eps=} {hidden_dim=}")

        # 1) 属性の設定

        super(CrossAttention, self).__init__()
        if hidden_dim is None:
            hidden_dim = dim

        assert hidden_dim % num_heads == 0, "dim must be divisible by num_heads"

        self.dim = dim
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        self.qkv_bias = qkv_bias

        # 2) モジュールの初期化

        # 入力からQを計算する線形層
        self.q_linear = nn.Linear(dim, hidden_dim, bias=qkv_bias)

        # 条件付けからKとVを計算する線形層
        self.kv_linear = nn.Linear(dim, hidden_dim*2, bias=qkv_bias)

        # 出力の線形層
        self.proj = nn.Linear(hidden_dim, dim, bias=qkv_bias)

        # Qの正規化層
        self.ln_q = create_norm(
            'np_layernorm', dim=hidden_dim, eps=norm_eps
        )

        # Kの正規化層
        self.ln_k = create_norm(
            'np_layernorm', dim=hidden_dim, eps=norm_eps
        )

    def forward(self, x: torch.Tensor, cond: torch.Tensor, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        """
        マルチヘッド・クロスアテンションの順伝播

        Args:
            x: 入力テンソル
            cond: 条件付けテンソル
            mask: マスクテンソル（未使用）
        Returns:
            torch.Tensor: 出力テンソル
        """

        # 1) クエリ、キー、バリューの計算

        # バッチサイズ、シーケンス長、チャンネル数
        B, N, C = x.shape

        # クエリの計算
        # (B, N, C) -> (B, N, hidden_dim) -> (B, N, num_heads, head_dim)
        q = self.q_linear(x).reshape(B, N, self.num_heads, self.head_dim)

        # キーとバリューの計算
        # (B, M, C) -> (B, M, 2 * hidden_dim)
        # -> (B, M, 2, num_heads, head_dim)
        kv = self.kv_linear(cond).reshape(
            B, -1, 2, self.num_heads, self.head_dim
        )

        # KとVに分割
        # それぞれ (B, M, num_heads, head_dim)
        k, v = kv.unbind(2)

        # 2) QK-Normの適用

        # Qに正規化を適用
        # (B, N, num_heads, head_dim) -> (B, N, num_heads * head_dim)
        # -> レイヤー正規化 -> (B, N, num_heads, head
        q = self.ln_q(
            q.view(B, N, self.num_heads * self.head_dim)
        ).view(B, N, self.num_heads, self.head_dim).to(q.dtype)

        # Kに正規化を適用
        # (B, M, num_heads, head_dim) -> (B, M, num_heads * head_dim)
        # -> レイヤー正規化 -> (B, M, num_heads, head_dim)
        k = self.ln_k(
            k.view(B, -1, self.num_heads * self.head_dim)
        ).view(B, -1, self.num_heads, self.head_dim).to(k.dtype)

        # 3) スケールド・ドットプロダクト・アテンションの計算（FlashAttention使用）

        # (B, N, num_heads, head_dim) -> (B, num_heads, N, head_dim)
        # -> FlashAttention -> (B, num_heads, N, head_dim)
        # -> (B, N, num_heads, head_dim)
        x = torch.nn.functional.scaled_dot_product_attention(
            q.transpose(1, 2),
            k.transpose(1, 2),
            v.transpose(1, 2),
            is_causal=False
        ).transpose(1, 2).contiguous()
        
        # 4) 出力層の適用

        # ヘッドを結合
        # (B, N, num_heads, head_dim) -> (B, N, hidden_dim)
        x = x.view(B, -1, self.num_heads * self.head_dim)

        # 線形層を適用
        # (B, N, hidden_dim) -> (B, N, dim)
        x = self.proj(x)

        return x
    
    def custom_init(self, init_std: float) -> None:
        logger.info(f"CrossAttentionの重みをカスタム初期化 {init_std=}")

        for linear in (self.q_linear, self.kv_linear):
            nn.init.trunc_normal_(linear.weight, mean=0.0, std=0.02)

        nn.init.trunc_normal_(self.proj.weight, mean=0.0, std=init_std)

### MLP

In [ ]:
class Mlp(nn.Module):
    """
    MLP（Multi-Layer Perceptron）の実装

    - timmライブラリを参考 https://github.com/huggingface/pytorch-image-models/blob/main/timm/layers/mlp.py
    - Dropout層は含まれていない
    - Transformer内部ではなく、DiTの条件付けの処理で使用する
    """
    def __init__(
        self,
        in_features: int,
        hidden_features: Optional[int] = None,
        out_features: Optional[int] = None,
        act_layer: Any = lambda: nn.GELU(approximate="tanh"),
        norm_layer: Optional[Any] = None,
        bias: bool = True,
    ):
        """
        Args:
            in_features: 入力テンソルの次元
            hidden_features: 隠れ層の特徴数。Noneの場合、in_featuresと同じ
            out_features: 出力特徴数。Noneの場合、in_featuresと同じ
            act_layer: 活性化関数のレイヤーコンストラクタ
            norm_layer: 正規化レイヤーのコンストラクタ。Noneの場合、Identityを使用
            bias: 線形層でバイアスを使用するかどうか
        """
        logger.info(f"MLPを初期化 {in_features=} {hidden_features=} {out_features=} {act_layer=} {norm_layer=} {bias=}")

        # 1) 属性の設定

        super().__init__()

        out_features = out_features or in_features
        hidden_features = hidden_features or in_features

        # 2) モジュールの初期化

        self.fc1 = nn.Linear(in_features, hidden_features, bias=bias)

        self.act = act_layer()

        self.norm = norm_layer if norm_layer is not None else nn.Identity()

        self.fc2 = nn.Linear(hidden_features, out_features, bias=bias)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.debug(f"MLPの順伝播 {x.shape=}")

        # 1) 最初の全結合層を適用し、中間次元に拡張
        x = self.fc1(x)
        logger.debug(f"MLP fc1後 {x.shape=}")

        # 2) 活性化関数を適用

        x = self.act(x)

        # 必要に応じて正規化層を適用
        x = self.norm(x)

        # 3) 2番目の全結合層を適用し、元の次元に戻す
        x = self.fc2(x)
        logger.debug(f"MLP fc2後 {x.shape=}")

        return x

In [ ]:
class FeedForward(nn.Module):
    """
    SiLU活性化関数を用いたフィードフォワードブロック

    - DiTBlock内部で使用されるFeed-forwardネットワーク
    - SwiGLU(SiLU Gated Linear Unit)を採用し高性能化
    - SwiGLUは3つの線形層を使うため、通常のFFNとパラメータ数が等しくなるように調整
    """
    def __init__(
        self,
        dim: int,
        hidden_dim: int,
        multiple_of: int,
        use_bias: bool,
    ):
        """
        Args:
            dim (int): 入力および出力の次元
            hidden_dim (int): 2つの線形層の間の隠れ次元
            multiple_of (int): 隠れ次元をこの値の最小公倍数に丸める
            use_bias (bool): 線形層でバイアス項を使用するかどうか
        """
        logger.info(f"FeedForwardブロックを初期化 {dim=} {hidden_dim=} {multiple_of=} {use_bias=}")

        super().__init__()

        # 1) 属性の設定

        self.dim = dim

        # 隠れ次元を2/3に調整
        hidden_dim = int(2 * hidden_dim / 3)
        logger.debug(f"FeedForward隠れ次元調整前 {hidden_dim=}")

        # ハードウェア効率のために、multiple_ofの最小公倍数に丸める
        self.hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)
        logger.debug(f"FeedForward隠れ次元調整後 {self.hidden_dim=}")
        
        # 2) モジュールの初期化 

        # ゲートとして機能する線形層
        self.w1 = nn.Linear(dim, self.hidden_dim, bias=use_bias)

        # 最初の線形層
        self.w2 = nn.Linear(dim, self.hidden_dim, bias=use_bias)

        # 出力の線形層
        self.w3 = nn.Linear(self.hidden_dim, dim, bias=use_bias)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        FeedForwardブロックの順伝播

        Args:
            x (torch.Tensor): 入力テンソル
        Returns:
            torch.Tensor: 出力テンソル
        """
        logger.debug(f"FeedForwardの順伝播 {x.shape=}")

        # ゲートの出力にSiLU活性化関数を適用し、もう一方の線形層の出力と要素ごとに乗算
        # 最後に出力線形層を適用
        return self.w3(F.silu(self.w1(x)) * self.w2(x))

    def custom_init(self, init_std: float) -> None:
        logger.info(f"FeedForwardの重みをカスタム初期化 {init_std=}")

        nn.init.trunc_normal_(self.w1.weight, mean=0.0, std=0.02)
        for linear in (self.w2, self.w3):
            nn.init.trunc_normal_(linear.weight, mean=0.0, std=init_std)

In [ ]:
class FeedForwardECMoe(nn.Module):
    """
    Expert choice routingを使用したMoEベースのフィードフォワード層

    - 専門家が、自分が得意とするトークン上位k個を選ぶことで、負荷集中を回避
    - GELU活性化関数を使用
    """
    def __init__(
        self,
        num_experts: int,
        expert_capacity: float,
        dim: int,
        hidden_dim: int,
        multiple_of: int,
    ):
        """
        Args:
            num_experts (int): レイヤー内の専門家の数
            expert_capacity (float): 専門家あたりのトークン数を決定する容量係数
            dim (int): 入力および出力の次元
            hidden_dim (int): 2つの線形層の間の隠れ次元
            multiple_of (int): 隠れ次元をこの値の最小公倍数に丸める
        """
        logger.info(f"FeedForwardECMoeを初期化 {num_experts=} {expert_capacity=} {dim=} {hidden_dim=} {multiple_of=}")

        super().__init__()

        # 1) 属性の設定

        self.num_experts = num_experts

        self.expert_capacity = expert_capacity

        self.dim = dim

        # ハードウェア効率のために、multiple_ofの最小公倍数に丸める
        self.hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)

        # 2) モジュールの初期化

        # 専門家ごとに異なる重みを作成
        self.w1 = nn.Parameter(
            torch.ones(num_experts, dim, self.hidden_dim)
        )

        # 専門家ごとに異なる重みを作成
        self.w2 = nn.Parameter(
            torch.ones(num_experts, self.hidden_dim, dim)
        )

        self.gate = nn.Linear(dim, num_experts, bias=False)

        self.gelu = nn.GELU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        FeedForwardECMoeの順伝播

        Args:
            x (torch.Tensor): 入力テンソル
        Returns:
            torch.Tensor: 出力テンソル
        """
        logger.info(f"FeedForwardECMoeの順伝播 {x.shape=}")

        assert len(x.shape) == 3

        # (batch_size, seq_len, dim)
        n, t, d = x.shape

        # 専門家あたりのトークン数を計算
        tokens_per_expert = int(
            self.expert_capacity * t / self.num_experts
        )
        logger.debug(f"{tokens_per_expert=}")

        # 1) ゲーティング（各専門家との関連度スコアの計算）

        # ロジットを計算
        # (batch_size, seq_len, dim) -> (batch_size, seq_len, num_experts)
        scores = self.gate(x)

        # Softmaxで確率に変換
        # (batch_size, seq_len, num_experts)
        probs = F.softmax(scores, dim=-1)

        # 2) ルーティング

        # インデックスmとスコアgを取得
        # (batch_size, seq_len, num_experts)
        # -> (batch_size, num_experts, tokens_per_expert)
        # -> (batch_size, num_experts, tokens_per_expert),
        #    (batch_size, num_experts, tokens_per_expert)
        g, m = torch.topk(
            probs.permute(0, 2, 1),
            tokens_per_expert,
            dim=-1
        )

        # 3) 専門家の適用

        # ワンホットエンコーディングでルーティングマスクを作成
        # (batch_size, num_experts, top_k, seq_len)
        p = F.one_hot(m, num_classes=t).float()
        logger.debug(f"ルーティングマスク {p.shape=}")

        # 選ばれたトークンを各専門家に分配
        # (batch_size, num_experts, top_k, dim)
        xin = torch.einsum('nekt, ntd -> nekd', p, x)
        logger.debug(f"専門家への分配 {xin.shape=}")

        # 各専門家でフィードフォワードネットワークを適用
        # (batch_size, num_experts, top_k, 4 * dim)
        h = torch.einsum('nekd, edf -> nekf', xin, self.w1)
        logger.debug(f"専門家の最初の線形層後 {h.shape=}")

        # GELU活性化関数を適用
        h = self.gelu(h)

        # 出力線形層を適用
        # (batch_size, num_experts, top_k, dim)
        h = torch.einsum('nekf, efd -> nekd', h, self.w2)
        logger.debug(f"専門家の出力線形層後 {h.shape=}")

        # 4) 出力の集約

        # 専門家の出力を重み付けして集約
        # (batch_size, num_experts, top_k, dim)
        out = g.unsqueeze(dim=-1) * h
        logger.debug(f"重み付け後 {out.shape=}")

        # (batch_size, seq_len, dim)
        out = torch.einsum('nekt, nekd -> ntd', p, out)
        logger.debug(f"最終出力 {out.shape=}")

        return out
    
    def custom_init(self, init_std: float):
        logger.info(f"FeedForwardECMoeの重みをカスタム初期化 {init_std=}")
        nn.init.trunc_normal_(self.gate.weight, mean=0.0, std=0.02)
        nn.init.trunc_normal_(self.w1, mean=0.0, std=0.02)
        nn.init.trunc_normal_(self.w2, mean=0.0, std=init_std)


### TimestepEmbedder

In [ ]:
class TimestepEmbedder(nn.Module):
    """
    拡散モデルのタイムステップを埋め込むモジュール
    DiTのAdaLNのタイムステップ条件付けに使用される
    """
    def __init__(
        self,
        hidden_size: int,
        act_layer: Any,
        frequency_embedding_size: int = 512
    ):
        """
        Args:
            hidden_size (int): 隠れ次元のサイズ
            act_layer (Any): 活性化レイヤーのコンストラクタ
            frequency_embedding_size (int): 周波数埋め込みのサイズ
        """
        super().__init__()

        # 1) モジュールの初期化

        # MLPの構築
        self.mlp = nn.Sequential(
            nn.Linear(frequency_embedding_size, hidden_size, bias=True),
            act_layer(),
            nn.Linear(hidden_size, hidden_size, bias=True),
        )

        # 2) 属性の設定

        self.frequency_embedding_size = frequency_embedding_size

    @staticmethod
    def timestep_embedding(t: torch.Tensor, dim: int, max_period: int = 10000) -> torch.Tensor:
        """
        正弦波埋め込みを作成する

        Args:
            t (torch.Tensor): タイムステップテンソル
            dim (int): 埋め込みの次元
            max_period (int): 周波数の最大周期
        Returns:
            torch.Tensor: タイムステップの埋め込みテンソル
        """
        logger.info(f"タイムステップ埋め込みを作成 {t.shape=} {dim=} {max_period=}")

        half = dim // 2

        freqs = torch.exp(
            -math.log(max_period) * torch.arange(
                start=0,
                end=half,
                dtype=torch.float32,
                device=t.device
            ) / half
        )
        logger.debug(f"{freqs=}")

        args = t[:, None].float() * freqs[None]

        # 正弦波と余弦波を計算して結合
        embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)

        # 次元が奇数の場合、ゼロパディングを追加
        if dim % 2:
            embedding = torch.cat(
                [embedding, torch.zeros_like(embedding[:, :1])],
                dim=-1
            )

        logger.debug(f"タイムステップ埋め込みテンソル {embedding.shape=}")

        return embedding

    def forward(self, t: torch.Tensor) -> torch.Tensor:
        """
        タイムステップ埋め込みの順伝播

        Args:
            t (torch.Tensor): タイムステップテンソル
        Returns:
            torch.Tensor: 埋め込みテンソル
        """
        logger.debug(f"タイムステップ埋め込みの順伝播 {t.shape=}")

        # 1) タイムステップの周波数埋め込みを計算

        t_freq = self.timestep_embedding(
            t, self.frequency_embedding_size
        ).to(self.dtype)

        # 2) MLPを適用して最終的な埋め込みを取得

        return self.mlp(t_freq)

    @property
    def dtype(self) -> torch.dtype:
        return next(self.parameters()).dtype

### CaptionProjection

In [ ]:
class CaptionProjection(nn.Module):
    """
    テキストエンコーダーから得られたプロンプトの特徴量をDiT用に変換するモジュール
    内部ではMLPを使用
    
    Projects caption embeddings to model dimension.
    
    Args:
        in_channels (int): Number of input channels
        hidden_size (int): Size of hidden dimension
        act_layer (Any): Activation layer constructor
        norm_layer (Any): Normalization layer constructor
    """
    def __init__(
        self,
        in_channels: int,
        hidden_size: int,
        act_layer: Any,
        norm_layer: Any
    ) -> None:
        """
        Args:
            in_channels (int): 入力チャンネル数
            hidden_size (int): 隠れ次元のサイズ
            act_layer (Any): 活性化レイヤーのコンストラクタ
            norm_layer (Any): 正規化レイヤーのコンストラクタ
        """
        logger.info(f"CaptionProjectionを初期化 {in_channels=} {hidden_size=} {act_layer=} {norm_layer=}")

        super().__init__()

        self.y_proj = Mlp(
            in_features=in_channels,
            hidden_features=hidden_size,
            out_features=hidden_size,
            act_layer=act_layer,
            norm_layer=norm_layer
        )
    
    def forward(self, caption: torch.Tensor) -> torch.Tensor:
        """
        CaptionProjectionの順伝播
        Args:
            caption (torch.Tensor): 入力キャプションテンソル
        Returns:
            torch.Tensor: 変換後のキャプションテンソル
        """
        logger.info(f"CaptionProjectionの順伝播 {caption.shape=}")
        result = self.y_proj(caption)
        logger.debug(f"変換後のキャプションテンソル {result.shape=}")
        return result

### AttentionBlockPromptEmbedding

In [ ]:
class AttentionBlockPromptEmbedding(nn.Module):
    """
    CaptionProjectionの出力を洗練するためのアテンションブロック

    - セルフアテンションとフィードフォワードネットワークで構成
    
    Args:
        dim (int): Input and output dimension
        head_dim (int): Dimension size per attention head
        mlp_ratio (float): Multiplier for feed-forward network hidden dimension w.r.t input dim
        multiple_of (int): Round feed-forward network hidden dimension up to nearest multiple of this value
        norm_eps (float): Epsilon value for layer normalization
        use_bias (bool): Whether to use bias terms in linear layers
    """
    def __init__(
        self,
        dim: int,
        head_dim: int, 
        mlp_ratio: float,
        multiple_of: int,
        norm_eps: float,
        use_bias: bool,
    ) -> None:
        """
        Args:
            dim (int): 入力および出力の次元
            head_dim (int): アテンションヘッドあたりの次元サイズ
            mlp_ratio (float): フィードフォワードネットワークの隠れ次元を入力次元に対して乗算する倍率
            multiple_of (int): フィードフォワードネットワークの隠れ次元をこの値の最小公倍数に丸める
            norm_eps (float): レイヤー正規化のイプシロン値
            use_bias (bool): 線形層でバイアス項を使用するかどうか
        """
        logger.info(f"AttentionBlockPromptEmbeddingを初期化 {dim=} {head_dim=} {mlp_ratio=} {multiple_of=} {norm_eps=} {use_bias=}")

        # 1) 属性の設定

        super().__init__()
        assert dim % head_dim == 0, 'Hidden dimension must be divisible by head dim'
        
        self.dim = dim

        self.num_heads = dim // head_dim
        logger.debug(f"{self.num_heads=}")

        # 2) モジュールの初期化
        
        # アテンション前のレイヤー正規化
        self.norm1 = create_norm('layernorm', dim, eps=norm_eps)

        # セルフアテンション層
        self.attn = SelfAttention(
            dim=dim,
            num_heads=self.num_heads,
            qkv_bias=use_bias,
            norm_eps=norm_eps,
        )

        # フィードフォワード前のレイヤー正規化
        self.norm2 = create_norm('layernorm', dim, eps=norm_eps)

        # フィードフォワード層
        self.mlp = FeedForward(
            dim=dim,
            hidden_dim=int(dim * mlp_ratio),
            multiple_of=multiple_of,
            use_bias=use_bias,
        )

    def forward(self, x: torch.Tensor, **kwargs) -> torch.Tensor:
        """
        AttentionBlockPromptEmbeddingの順伝播

        Args:
            x (torch.Tensor): 入力テンソル
        Returns:
            torch.Tensor: 出力テンソル
        """
        logger.info(f"AttentionBlockPromptEmbeddingの順伝播 {x.shape=}")
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

    def custom_init(self, init_std: float = 0.02) -> None:
        logger.info(f"AttentionBlockPromptEmbeddingの重みをカスタム初期化 {init_std=}")
        self.attn.custom_init(init_std)
        self.mlp.custom_init(init_std)

### DiTBlock

In [ ]:
class DiTBlock(nn.Module):
    """
    DiTを構成する単一のTransformerブロック

    画像パッチのシーケンスを入力とし、テキストyとタイムステップcによる条件付きの処理

    - 画像特徴量はSelf-Attentionで処理
    - テキスト条件付けはCross-Attentionで実現
    - タイムステップ条件付は、AdaLNでshiftとscaleとgateを予測することで実現
    - 通常のフィードフォワード層かMoEスタイルのフィードフォワード層を選択可能
    """
    def __init__(
        self,
        dim: int,
        head_dim: int,
        mlp_ratio: float,
        qkv_ratio: float,
        multiple_of: int,
        pooled_emb_dim: int,
        norm_eps: float,
        depth_init: bool,
        layer_id: int,
        num_layers: int,
        compress_xattn: bool,
        use_bias: bool,
        moe_block: bool,
        num_experts: int,
        expert_capacity: float,
    ):
        """
        Args:
            dim (int): ブロックの入力および出力の次元
            head_dim (int): アテンションヘッドあたりの次元
            mlp_ratio (float): MLPブロック内の線形層間の隠れ次元の倍率
            qkv_ratio (float): アテンションブロック内のqkv層の次元の倍率
            multiple_of (int): MLPブロック内の隠れ次元をこの値の最小公倍数に丸める
            pooled_emb_dim (int): プールされたキャプション埋め込みの次元
            norm_eps (float): レイヤー正規化のイプシロン
            depth_init (bool): MLP/アテンションブロックの最後の層の重みをブロックインデックスに基づいて初期化するかどうか
            layer_id (int): DiTモデル内のこのブロックのインデックス
            num_layers (int): DiTモデル内のブロックの総数
            compress_xattn (bool): qkv_ratioを使用してクロスアテンションのqkv次元をスケーリングするかどうか
            use_bias (bool): 線形層でバイアスを使用するかどうか
            moe_block (bool): MLPブロックで専門家選択型のMixture-of-Expertsを使用するかどうか
            num_experts (int): MoEブロックを使用する場合の専門家の数
            expert_capacity (float): MoEブロックを使用する場合の各専門家の容量係数
        """
        logger.info(f"DiTBlockを初期化 {dim=} {head_dim=} {mlp_ratio=} {qkv_ratio=} {multiple_of=} {pooled_emb_dim=} {norm_eps=} {depth_init=} {layer_id=} {num_layers=} {compress_xattn=} {use_bias=} {moe_block=} {num_experts=} {expert_capacity=}")

        super().__init__()

        # 1) 属性の設定

        self.dim = dim

        # qkv層の隠れ次元を計算
        qkv_hidden_dim = (
            (head_dim * 2) * ((int(dim * qkv_ratio) + head_dim * 2 - 1) // (head_dim * 2))
            if qkv_ratio != 1 else dim
        )
        logger.debug(f"{qkv_hidden_dim=}")

        # MLPの隠れ次元を計算
        mlp_hidden_dim = int(dim * mlp_ratio)
        logger.debug(f"{mlp_hidden_dim=}")

        # 2) モジュールの初期化

        # セルフアテンション前のレイヤー正規化
        self.norm1 = create_norm('layernorm', dim, eps=norm_eps)

        # セルフアテンション層
        self.attn = SelfAttention(
            dim=dim,
            num_heads=qkv_hidden_dim // head_dim,
            qkv_bias=use_bias,
            norm_eps=norm_eps,
            hidden_dim=qkv_hidden_dim,
        )

        # テキスト条件付け用のクロスアテンション層
        self.cross_attn = CrossAttention(
            dim=dim,
            num_heads=qkv_hidden_dim // head_dim if compress_xattn else dim // head_dim,
            qkv_bias=use_bias,
            norm_eps=norm_eps,
            hidden_dim=qkv_hidden_dim if compress_xattn else dim,
        )

        # クロスアテンション前のレイヤー正規化
        self.norm2 = create_norm('layernorm', dim, eps=norm_eps)

        # MLP前のレイヤー正規化
        self.norm3 = create_norm('layernorm', dim, eps=norm_eps)
        
        # フィードフォワード層（通常またはMoE）
        self.mlp = (
            FeedForwardECMoe(num_experts, expert_capacity, dim, mlp_hidden_dim, multiple_of)
            if moe_block else
            FeedForward(dim, mlp_hidden_dim, multiple_of, use_bias)
        )

        # AdaLNのシフトとスケールとゲートを予測するモジュール
        self.adaLN_modulation = nn.Sequential(
            nn.GELU(approximate="tanh"),
            nn.Linear(pooled_emb_dim, 6 * dim, bias=True),
        )
        
        # 重みの初期化標準偏差を計算
        self.weight_init_std = (
            0.02 / (2 * (layer_id + 1)) ** 0.5 if depth_init else
            0.02 / (2 * num_layers) ** 0.5
        )
        
    def forward(self, x: torch.Tensor, y: torch.Tensor, c: torch.Tensor, **kwargs) -> torch.Tensor:
        """
        DiTBlockの順伝播

        Args:
            x (torch.Tensor): 画像パッチのシーケンステンソル
            y (torch.Tensor): テキスト条件付けテンソル
            c (torch.Tensor): タイムステップ条件付けテンソル
        Returns:
            torch.Tensor: 出力テンソル
        """
        logger.info(f"DiTBlockの順伝播 {x.shape=} {y.shape=} {c.shape=}")

        # 1) AdaLNのシフト、スケール、ゲートを予測

        shift_msa, scale_msa, gate_msa, shift_mlp, scale_mlp, gate_mlp = (
            self.adaLN_modulation(c).chunk(6, dim=1)
        )
        logger.debug(f"{shift_msa.shape=} {scale_msa.shape=} {gate_msa.shape=} {shift_mlp.shape=} {scale_mlp.shape=} {gate_mlp.shape=}")

        # 2) Self-Attention

        # レイヤー正規化
        # -> AdaLNのshiftとscaleを適用
        # -> セルフアテンションを適用
        # -> AdaLNのgateを適用
        # -> 残差接続を適用
        x = x + gate_msa.unsqueeze(1) * self.attn(modulate(self.norm1(x), shift_msa, scale_msa))

        # 3) Cross-Attention

        # レイヤー正規化
        # -> テキスト条件付けを用いたクロスアテンションを適用
        # -> 残差接続を適用
        x = x + self.cross_attn(self.norm2(x), y)

        # 4) Feed-Forwardネットワーク

        # レイヤー正規化
        # -> AdaLNのshiftとscaleを適用
        # -> フィードフォワードネットワークを適用
        # -> AdaLNのgateを適用
        # -> 残差接続を適用
        x = x + gate_mlp.unsqueeze(1) * self.mlp(modulate(self.norm3(x), shift_mlp, scale_mlp))

        logger.debug(f"DiTBlock出力テンソル {x.shape=}")
        return x

    def custom_init(self):
        logger.info(f"DiTBlockの重みをカスタム初期化")

        for norm in (self.norm1, self.norm2, self.norm3):
            norm.reset_parameters()
        self.attn.custom_init(self.weight_init_std)
        self.cross_attn.custom_init(self.weight_init_std)
        self.mlp.custom_init(self.weight_init_std)

### T2IFinalLayer

In [ ]:
class T2IFinalLayer(nn.Module):
    """
    DiTモデルの最終出力層

    - Transformerブロックの出力を画像パッチに変換する
    - 出力直前に、タイムステップ情報cを用いたAdaLNを適用する
    """

    def __init__(
        self,
        hidden_size: int,
        time_emb_dim: int,
        patch_size: int,
        out_channels: int,
        act_layer: Any,
        norm_final: nn.Module
    ):
        """
        Args:
            hidden_size (int): 隠れ次元のサイズ
            time_emb_dim (int): タイムステップ埋め込みの次元
            patch_size (int): 画像パッチのサイズ
            out_channels (int): 出力チャンネル数
            act_layer (Any): 活性化レイヤーのコンストラクタ
            norm_final (nn.Module): 最終正規化レイヤー
        """
        logger.info(f"T2IFinalLayerを初期化 {hidden_size=} {time_emb_dim=} {patch_size=} {out_channels=} {act_layer=} {norm_final=}")

        super().__init__()

        # 1) モジュールの初期化

        # 隠れ次元から画像パッチへの線形変換
        self.linear = nn.Linear(
            hidden_size,
            patch_size * patch_size * out_channels,
            bias=True
        )

        # タイムステップ埋め込みからAdaLNのshiftとscaleを予測するモジュール
        self.adaLN_modulation = nn.Sequential(
            act_layer(),
            nn.Linear(time_emb_dim, 2 * hidden_size, bias=True)
        )

        # 最終正規化レイヤー
        self.norm_final = norm_final

    def forward(self, x: torch.Tensor, c: torch.Tensor) -> torch.Tensor:
        """
        T2IFinalLayerの順伝播
        Args:
            x (torch.Tensor): 入力テンソル
            c (torch.Tensor): タイムステップ埋め込みテンソル
        Returns:
            torch.Tensor: 出力テンソル
        """
        logger.info(f"T2IFinalLayerの順伝播 {x.shape=} {c.shape=}")

        # AdaLNのshiftとscaleを予測
        shift, scale = self.adaLN_modulation(c).chunk(2, dim=1)

        # レイヤー正規化を適用し、AdaLNのshiftとscaleを適用
        x = modulate(self.norm_final(x), shift, scale)

        # 線形層を適用して画像パッチに変換
        x = self.linear(x)
        logger.debug(f"T2IFinalLayer出力テンソル {x.shape=}")

        return x

### DiT

In [ ]:
def get_1d_sincos_pos_embed_from_grid(embed_dim: int, pos: np.ndarray) -> np.ndarray:
    """
    1次元の正弦波位置埋め込みを生成する
    Attention is All You Need論文の手法に基づく位置埋め込み

    Args:
        embed_dim (int): 埋め込みの次元
        pos (np.ndarray): 位置インデックスの配列 (M,)
    Returns:
        np.ndarray: 位置埋め込みテンソル (M, D)
    """
    logger.info(f"1D正弦波位置埋め込みを生成 {embed_dim=} {pos.shape=}")

    assert embed_dim % 2 == 0

    # 1) 周波数（omega）の計算

    omega = np.arange(embed_dim // 2, dtype=np.float64)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # (D/2,)

    # 2) 位置と周波数の積を計算

    pos = pos.reshape(-1)  # (M,)
    logger.debug(f"位置インデックス reshaped {pos.shape=}")

    # 全ての位置と周波数の組み合わせの積を計算
    out = np.einsum('m,d->md', pos, omega)  # (M, D/2), outer product
    logger.debug(f"位置と周波数の積 {out.shape=}")

    # 3) 正弦波と余弦波を計算して結合

    emb_sin = np.sin(out)  # (M, D/2)
    emb_cos = np.cos(out)  # (M, D/2)

    result = np.concatenate([emb_sin, emb_cos], axis=1)
    return result  # (M, D)

In [ ]:
def get_2d_sincos_pos_embed_from_grid(embed_dim: int, grid: np.ndarray) -> np.ndarray:
    """
    2次元の正弦波位置埋め込みを生成する
    画像パッチの位置情報をエンコードするために使用

    Args:
        embed_dim (int): 埋め込みの次元
        grid (np.ndarray): 位置グリッドの配列 (2, H, W)
    Returns:
        np.ndarray: 位置埋め込みテンソル (H*W, D)
    """
    logger.info(f"2D正弦波位置埋め込みを生成 {embed_dim=} {grid.shape=}")

    assert embed_dim % 2 == 0

    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[0])
    logger.debug(f"水平位置埋め込み {emb_h.shape=}")

    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[1])
    logger.debug(f"垂直位置埋め込み {emb_w.shape=}")

    result = np.concatenate([emb_h, emb_w], axis=1)
    return result  # (H*W, D)

In [ ]:
def ntuple(n: int):
    """
    単一の値を指定された長さのタプルに変換する関数を生成するための関数
    get_2d_sincos_pos_embed_from_grid関数で使用される

    Args:
        n (int): タプルの長さ
    Returns:
        Callable: 入力をn-タプルに変換する関数
    """
    def parse(x):
        if isinstance(x, Iterable) and not isinstance(x, str):
            return x
        return tuple(repeat(x, n))

    return parse

In [ ]:
def get_2d_sincos_pos_embed(
    embed_dim: int,
    grid_size: Union[int, Tuple[int, int]],
    cls_token: bool = False,
    extra_tokens: int = 0,
    pos_interp_scale: float = 1.0,
    base_size: int = 16
) -> np.ndarray:
    """
    画像のパッチの2次元正弦波位置埋め込みを生成する

    Args:
        embed_dim (int): 埋め込みの次元
        grid_size (Union[int, Tuple[int, int]]): グリッドのサイズ（整数またはタプル）
        cls_token (bool): クラストークン用の位置埋め込みを含めるかどうか
        extra_tokens (int): 追加のトークン用の位置埋め込み数
        pos_interp_scale (float): 位置埋め込みの補間スケール
        base_size (int): 基準となるグリッドサイズ
    Returns:
        np.ndarray: 位置埋め込みテンソル (N, D)
    """
    logger.info(f"2D正弦波位置埋め込みを生成 {embed_dim=} {grid_size=} {cls_token=} {extra_tokens=} {pos_interp_scale=} {base_size=}")

    # 1) グリッドサイズの正規化

    to_2tuple = ntuple(2)

    # 例えば入力が32の場合、(32, 32)に変換
    if isinstance(grid_size, int):
        grid_size = to_2tuple(grid_size)

    # Interpolate position embeddings to adapt model across resolutions. Interestingly, without any interpolation
    # the model does converge slowly at start (~1000 steps) but eventually achieves near similar qualitative performance.

    # 2) グリッド座標の計算とスケーリング（補完）

    # 解像度が変わっても同じ位置埋め込みを使用できるように、
    # base_sizeとpos_interp_scaleに基づいてグリッドをスケーリング

    grid_h = np.arange(grid_size[0], dtype=np.float32) / (grid_size[0] / base_size) / pos_interp_scale

    grid_w = np.arange(grid_size[1], dtype=np.float32) / (grid_size[1] / base_size) / pos_interp_scale

    # 3) メッシュグリッドを作成

    grid = np.meshgrid(grid_w, grid_h)

    grid = np.stack(grid, axis=0)

    grid = grid.reshape([2, 1, grid_size[1], grid_size[0]])

    # 4) 実際の埋め込みを生成

    # sin・cos関数で位置埋め込みを生成
    pos_embed = get_2d_sincos_pos_embed_from_grid(embed_dim, grid)

    # 5) 特殊トークンの処理

    # クラストークンや追加トークン用のゼロ埋め込みを追加
    if cls_token and extra_tokens > 0:
        pos_embed = np.concatenate(
            [np.zeros([extra_tokens, embed_dim]), pos_embed], axis=0
        )

    return pos_embed

In [ ]:
def mask_out_token(x: torch.Tensor, ids_keep: torch.Tensor) -> torch.Tensor:
    """
    画像パッチのシーケンスから、指定されたインデックスのトークンだけを残す

    Args:
        x (torch.Tensor): 入力テンソル (N, L, D)
        ids_keep (torch.Tensor): 残すトークンのインデックス (N, L_masked)
    Returns:
        torch.Tensor: マスク後のテンソル (N, L_masked, D)
    """
    logger.info(f"トークンをマスクアウト {x.shape=} {ids_keep.shape=}")

    N, L, D = x.shape  # batch, length, dim

    # ids_keepを特徴量の次元Dに合わせて拡張し、
    # gatherでids_keepのバッチだけを抜き出す
    x_masked = torch.gather(
        x,
        dim=1,
        index=ids_keep.unsqueeze(-1).repeat(1, 1, D)
    )
    logger.debug(f"マスク後のテンソル {x_masked.shape=}")

    return x_masked

In [ ]:
def get_mask(batch: int, length: int, mask_ratio: float, device: torch.device) -> Dict[str, torch.Tensor]:
    """
    画像パッチをランダムにマスクするためのインデックスとバイナリマスクを生成する

    torch.randpermでのシャッフルではなく、ノイズ生成+argsortで計算を効率化

    Args:
        batch (int): バッチサイズ
        length (int): シーケンスの長さ
        mask_ratio (float): マスクするトークンの割合
        device (torch.device): マスクテンソルを配置するデバイス
    Returns:
        Dict[str, torch.Tensor]: 
            - mask: バイナリマスク、0は保持、1は削除
            - ids_keep: 残すトークンのインデックス（mask_out_token関数で使用）
            - ids_restore: 元の順序を復元するためのインデックス（unmasking_tokensで使用）
    """
    logger.info(f"マスクを生成 {batch=} {length=} {mask_ratio=} {device=}")

    # 保持するトークン数を計算
    len_keep = int(length * (1 - mask_ratio))
    logger.debug(f"{len_keep=}")

    # ランダムなノイズを生成（0から1の一様分布）
    # 例: [0.8, 0.1, 0.5, 0.9]
    # (batch, length)
    noise = torch.rand(batch, length, device=device)

    # ノイズを昇順にソートしてインデックスを取得
    # 例: [1, 2, 0, 3]
    ids_shuffle = torch.argsort(noise, dim=1)

    # 元の順序を復元するためのインデックスを計算
    # 例: [2, 0, 1, 3]
    ids_restore = torch.argsort(ids_shuffle, dim=1)

    # 先頭からlen_keep個のインデックスを取得
    # 例: [1, 2]
    ids_keep = ids_shuffle[:, :len_keep]

    # バイナリマスクを1で初期化（0:保持、1:削除）
    # (batch, length)
    # 例: [1, 1, 1, 1]
    mask = torch.ones([batch, length], device=device)

    # 先頭からlen_keep個のトークンを保持（0の保持に設定）
    # 例: [0, 0, 1, 1]
    mask[:, :len_keep] = 0

    # 元の順序にマスクを復元
    # gatherはids_restoreに基づいてmaskを並び替える
    # 例: [1, 0, 0, 1]
    mask = torch.gather(mask, dim=1, index=ids_restore)

    return {
        'mask': mask,
        'ids_keep': ids_keep,
        'ids_restore': ids_restore
    }

In [ ]:
def unmask_tokens(x: torch.Tensor, ids_restore: torch.Tensor, mask_token: torch.Tensor) -> torch.Tensor:
    """
    マスキングされて処理されたトークン列に、マスクトークンを補完し、元の画像の順序を復元する

    Args:
        x (torch.Tensor): マスク後のトークン列 (N, L_masked, D)
        ids_restore (torch.Tensor): 元の順序を復元するためのインデックス (N, L)
        mask_token (torch.Tensor): マスクトークン (1, 1, D)
    Returns:
        torch.Tensor: 元の順序に復元されたトークン列 (N, L, D)
    """
    logger.info(f"マスクトークンを補完して元の順序を復元 {x.shape=} {ids_restore.shape=} {mask_token.shape=}")

    # 不足している分のマスクトークンを作成
    mask_tokens = mask_token.repeat(
        x.shape[0],
        ids_restore.shape[1] - x.shape[1],
        1
    )
    logger.debug(f"補完用マスクトークンテンソル {mask_tokens.shape=}")

    # 処理済みのトークンの後ろにマスクトークンを連結
    x_ = torch.cat([x, mask_tokens], dim=1)
    logger.debug(f"マスクトークンを連結後のテンソル {x_.shape=}")

    # 元の順序に復元
    x_ = torch.gather(
        x_,
        dim=1,
        index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2])
    )  # unshuffle

    return x_

In [ ]:
class DiT(nn.Module):
    """
    画像の潜在変数の入力と、タイムステップ・テキストを条件付けとして受け取り、
    ノイズ除去された画像の潜在変数を出力する

    - Patched Mixerを実装
    - Mixture-of-Experts (MoE) FFNレイヤーに対応

    
    Args:
        input_size (int, default: 32): Size of input image (assumed square)
        patch_size (int, default: 2): Size of patches for patch embedding
        in_channels (int, default: 4): Number of input image channels (by default assuming four channel latent space)
        dim (int, default: 1152): Dimension of transformer backbone, i.e., dimension of major transformer layers
        depth (int, default: 28): Number of transformer blocks
        head_dim (int, default: 64): Dimension of each attention head
        multiple_of (int, default: 256): Round hidden dimensions up to nearest multiple of this value in MLP block
        caption_channels (int, default: 4096): Number of channels in caption embeddings
        pos_interp_scale (float, default: 1.0): Scale for positional embedding interpolation (1.0 for 256x256, 2.0 for 512x512)
        norm_eps (float, default: 1e-6): Epsilon for layer normalization
        depth_init (bool, default: True): Whether to use depth-dependent initialization in DiT blocks
        qkv_multipliers (List[float], default: [1.0]): Multipliers for QKV projection dimensions in DiT blocks
        ffn_multipliers (List[float], default: [4.0]): Multipliers for FFN hidden dimensions in DiT blocks
        use_patch_mixer (bool, default: True): Whether to use patch mixer layers
        patch_mixer_depth (int, default: 4): Number of patch mixer blocks
        patch_mixer_dim (int, default: 512): Dimension of patch-mixer layers
        patch_mixer_qkv_ratio (float, default: 1.0): Multipliers for QKV projection dimensions in patch-mixer blocks
        patch_mixer_mlp_ratio (float, default: 1.0): Multipliers for FFN hidden dimensions in patch-mixer blocks
        use_bias (bool, default: True): Whether to use bias in linear layers
        num_experts (int, default: 8):  Number of experts if using MoE block
        expert_capacity (int, default: 1): Capacity factor for each expert if using MoE FFN layers
        experts_every_n (int, default: 2): Add MoE FFN layers every n blocks
    """
    def __init__(
        self,
        input_size: int = 32,
        patch_size: int = 2,
        in_channels: int = 4,
        dim: int = 1152,
        depth: int = 28,
        head_dim: int = 64,
        multiple_of: int = 256,
        caption_channels: int = 1024,
        pos_interp_scale: float = 1.0,
        norm_eps: float = 1e-6,
        depth_init: bool = True,
        qkv_multipliers: List[float] = [1.0],
        ffn_multipliers: List[float] = [4.0],
        use_patch_mixer: bool = True,
        patch_mixer_depth: int = 4,
        patch_mixer_dim: int = 512,
        patch_mixer_qkv_ratio: float = 1.0,
        patch_mixer_mlp_ratio: float = 1.0,
        use_bias: bool = True,
        num_experts: int = 8,
        expert_capacity: int = 1,
        experts_every_n: int = 2
    ):
        """
        Args:
            input_size (int, default: 32): 入力画像のサイズ（正方形と仮定）
            patch_size (int, default: 2): パッチ埋め込みのパッチサイズ
            in_channels (int, default: 4): 入力画像のチャンネル数（デフォルトでは4チャネルの潜在空間を想定）
            dim (int, default: 1152): トランスフォーマーバックボーンの次元、すなわち主要なトランスフォーマーレイヤーの次元
            depth (int, default: 28): トランスフォーマーブロックの数
            head_dim (int, default: 64): 各アテンションヘッドの次元
            multiple_of (int, default: 256): MLPブロック内の隠れ次元をこの値の最小公倍数に丸める
            caption_channels (int, default: 4096): キャプション埋め込みのチャンネル数
            pos_interp_scale (float, default: 1.0): 位置埋め込み補間のスケール（256x256の場合は1.0、512x512の場合は2.0）
            norm_eps (float, default: 1e-6): レイヤー正規化のイプシロン
            depth_init (bool, default: True): DiTブロックで深さ依存の初期化を使用するかどうか
            qkv_multipliers (List[float], default: [1.0]): DiTブロック内のQKV投影次元の乗数
            ffn_multipliers (List[float], default: [4.0]): DiTブロック内のFFN隠れ次元の乗数
            use_patch_mixer (bool, default: True): パッチミキサーレイヤーを使用するかどうか
            patch_mixer_depth (int, default: 4): パッチミキサーブロックの数
            patch_mixer_dim (int, default: 512): パッチミキサーレイヤーの次元
            patch_mixer_qkv_ratio (float, default: 1.0): パッチミキサーブロック内のQKV投影次元の乗数
            patch_mixer_mlp_ratio (float, default: 1.0): パッチミキサーブロック内のFFN隠れ次元の乗数
        """
        logger.info(f"DiTを初期化 {input_size=} {patch_size=} {in_channels=} {dim=} {depth=} {head_dim=} {multiple_of=} {caption_channels=} {pos_interp_scale=} {norm_eps=} {depth_init=} {qkv_multipliers=} {ffn_multipliers=} {use_patch_mixer=} {patch_mixer_depth=} {patch_mixer_dim=} {patch_mixer_qkv_ratio=} {patch_mixer_mlp_ratio=} {use_bias=} {num_experts=} {expert_capacity=} {experts_every_n=}")

        super().__init__()

        # 1) 基本的な属性の設定

        self.input_size = input_size
        self.in_channels = in_channels
        self.out_channels = in_channels
        self.patch_size = patch_size
        self.head_dim = head_dim
        self.pos_interp_scale = pos_interp_scale
        self.use_patch_mixer = use_patch_mixer

        # 2) 埋め込み層の初期化
        
        approx_gelu = lambda: nn.GELU(approximate="tanh")

        # 画像パッチ埋め込み層
        self.x_embedder = PatchEmbed(
            input_size, patch_size, in_channels, dim, bias=True
        )

        # タイムステップ埋め込み層
        self.t_embedder = TimestepEmbedder(dim, approx_gelu)

        # 基本グリッドサイズの計算
        self.base_size = input_size // self.patch_size
        logger.debug(f"{self.base_size=}")

        # パッチの位置埋め込みを学習可能なパラメータとして登録
        num_patches = self.x_embedder.num_patches
        self.register_buffer("pos_embed", torch.zeros(1, num_patches, dim))

        # キャプションの埋め込みと前処理を行う層
        self.y_embedder = CaptionProjection(
            in_channels=caption_channels,
            hidden_size=dim,
            act_layer=approx_gelu,
            norm_layer=create_norm('layernorm', dim, eps=norm_eps)
        )

        # テキスト埋め込みの事前処理層
        self.y_emb_preprocess = AttentionBlockPromptEmbedding(
            dim,
            head_dim,
            mlp_ratio=4.0,
            multiple_of=multiple_of,
            norm_eps=norm_eps,
            use_bias=use_bias
        )
        
        # プーリングされたテキスト特徴量をタイムステップに加算するために使用
        self.pooled_y_emb_process = Mlp(
            dim,
            dim,
            dim,
            approx_gelu,
            norm_layer=create_norm('layernorm', dim, eps=norm_eps)
        )

        # 3) Patch-Mixerブロックの構築

        # Patch-Mixerを使用する場合
        if self.use_patch_mixer:

            # MoEブロックを配置するインデックスを計算
            expert_blocks_idx = [
                i for i in range(1, patch_mixer_depth) 
                if (i+1) % experts_every_n == 0
            ]
            logger.debug(f"{expert_blocks_idx=}")

            # MoEブロックのフラグリストを作成
            is_moe_block = [
                True if i in expert_blocks_idx else False 
                for i in range(patch_mixer_depth)
            ]
            logger.debug(f"{is_moe_block=}")
                 
            # Patch-Mixerブロックのモジュールリストを作成
            self.patch_mixer = nn.ModuleList([
                DiTBlock(
                    dim=patch_mixer_dim,
                    head_dim=head_dim,
                    mlp_ratio=patch_mixer_mlp_ratio,
                    qkv_ratio=patch_mixer_qkv_ratio,
                    multiple_of=multiple_of,
                    pooled_emb_dim=dim,
                    norm_eps=norm_eps,
                    depth_init=False,
                    layer_id=0,
                    num_layers=depth,
                    compress_xattn=False,
                    use_bias=use_bias,
                    moe_block=is_moe_block[i],
                    num_experts=num_experts,
                    expert_capacity=expert_capacity
                ) for i in range(patch_mixer_depth)
            ])

            # Patch-Mixerの次元が入力次元と異なる場合
            if patch_mixer_dim != dim:

                # 入力次元からPatch-Mixer次元へのマッピング層
                self.patch_mixer_map_xin = nn.Sequential(
                    create_norm('layernorm', dim, eps=norm_eps),
                    nn.Linear(dim, patch_mixer_dim, bias=use_bias)
                )

                # Patch-Mixer次元から出力次元へのマッピング層
                self.patch_mixer_map_xout = nn.Sequential(
                    create_norm('layernorm', patch_mixer_dim, eps=norm_eps),
                    nn.Linear(patch_mixer_dim, dim, bias=use_bias)
                )

                # テキスト埋め込みからPatch-Mixer次元へのマッピング層
                self.patch_mixer_map_y = nn.Sequential(
                    create_norm('layernorm', dim, eps=norm_eps),
                    nn.Linear(dim, patch_mixer_dim, bias=use_bias)
                )

            # 同じ場合
            else:
                # そのまま通す恒等マッピング
                self.patch_mixer_map_xin = nn.Identity()
                self.patch_mixer_map_xout = nn.Identity()
                self.patch_mixer_map_y = nn.Identity()

        # 4) メインバックボーンの構築

        assert len(ffn_multipliers) == len(qkv_multipliers)

        # FFNの乗数がdepthと同じ場合
        if len(ffn_multipliers) == depth:
            # QKVとFFNの乗数をそのまま使用
            qkv_ratios = qkv_multipliers
            mlp_ratios = ffn_multipliers

        # それ以外の場合は、乗数を各パーティションに分配
        else:
            num_splits = len(ffn_multipliers)

            assert depth % num_splits == 0, 'number of blocks should be divisible by number of splits'

            depth_per_split = depth // num_splits

            qkv_ratios = list(np.array([
                [m]*depth_per_split for m in qkv_multipliers
            ]).reshape(-1))

            mlp_ratios = list(np.array([
                [m]*depth_per_split for m in ffn_multipliers
            ]).reshape(-1))

        # MoEブロックを配置するインデックスを計算
        # 最後のブロックにはMoEを使用しない
        expert_blocks_idx = [
            i for i in range(0, depth - 1) 
            if (i+1) % experts_every_n == 0
        ]

        # MoEブロックのフラグリストを作成
        is_moe_block = [
            True if i in expert_blocks_idx else False 
            for i in range(depth)
        ]
        
        # DitBlockのモジュールリストを作成
        self.blocks = nn.ModuleList([
            DiTBlock(
                dim=dim,
                head_dim=head_dim,
                mlp_ratio=mlp_ratios[i],
                qkv_ratio=qkv_ratios[i],
                multiple_of=multiple_of,
                pooled_emb_dim=dim,
                norm_eps=norm_eps,
                depth_init=depth_init,
                layer_id=i,
                num_layers=depth,
                compress_xattn=False,
                use_bias=use_bias,
                moe_block=is_moe_block[i],
                num_experts=num_experts,
                expert_capacity=expert_capacity
            ) for i in range(depth)
        ])
        
        # 5) マスクトークンと出力層

        # マスクされた部分を保管するためのトークンをゼロベクトルで登録
        self.register_buffer(
            "mask_token", 
            torch.zeros(1, 1, patch_size ** 2 * self.out_channels)
        )

        # 最終出力層（Transformer特徴量 -> 画像パッチ）
        self.final_layer = T2IFinalLayer(
            dim,
            dim,
            patch_size,
            self.out_channels,
            approx_gelu,
            create_norm('layernorm', dim, eps=norm_eps)
        )

        # 6) 重みの初期化

        self.initialize_weights()

    def forward_without_cfg(
        self,
        x: torch.Tensor,
        t: torch.Tensor,
        y: torch.Tensor,
        mask_ratio: float = 0,
        **kwargs
    ) -> dict:
        """
        Classifier-Free Guidance（CFG）を使用しない場合の本来の順伝播

        Args:
            x: 入力テンソル (batch_size, channels, height, width)
            t: タイムステップテンソル (batch_size,)
            y: キャプション埋め込みテンソル (batch_size, 1, seq_len, dim)
            mask_ratio: トレーニング中にマスクするパッチの割合（0から1の間）

        Returns:
            dict: 以下を含む辞書:
                - 'sample': 出力テンソル (batch_size, out_channels, height, width)
                - 'mask': マスキングが適用された場合のオプションのバイナリマスクテンソル、そうでなければNone
        """
        logger.info(f"DiTの順伝播（CFGなし） {x.shape=} {t.shape=} {y.shape=} {mask_ratio=}")

        # 1) 埋め込みと前処理

        self.h = x.shape[-2] // self.patch_size
        logger.debug(f"{self.h=}")

        self.w = x.shape[-1] // self.patch_size
        logger.debug(f"{self.w=}")

        # timm.PatchEmbedで画像をパッチに分割し埋め込み、位置埋め込みを加算
        # 更に位置埋め込みを加算
        # (num_batch, num_patches, dim)
        # num_patches = (H / patch_size) * (W / patch_size)
        x = self.x_embedder(x) + self.pos_embed
        logger.debug(f"パッチ埋め込み後のテンソル {x.shape=}")

        # タイムステップ埋め込み
        # 現在のタイムステップを全バッチに拡張し、TimestepEmbedderを適用
        t = self.t_embedder(t.expand(x.shape[0]))  # (N, D)
        logger.debug(f"タイムステップ埋め込み後のテンソル {t.shape=}")

        # CaptionProjectionでテキストを埋め込み
        # (N, 1, L, D)
        y = self.y_embedder(y)
        logger.debug(f"キャプション埋め込み後のテンソル {y.shape=}")

        # AttentionBlockPromptEmbeddingでテキスト埋め込みを前処理
        # (N, 1, L, D) -> (N, D)
        y = self.y_emb_preprocess(y.squeeze(dim=1)).unsqueeze(dim=1)
        logger.debug(f"前処理後のテキスト埋め込みテンソル {y.shape=}")  

        # プーリングされたテキスト埋め込みをタイムステップに加算
        y_pooled = self.pooled_y_emb_process(y.mean(dim=-2).squeeze(dim=1))
        t = t + y_pooled
        logger.debug(f"テキスト埋め込みを加算後のタイムステップテンソル {t.shape=}")

        mask = None

        # 2) Patch-Mixerの適用

        if self.use_patch_mixer:

            # 入力をPatch-Mixerの次元に投影
            x = self.patch_mixer_map_xin(x)
            logger.debug(f"Patch-Mixer入力マッピング後のテンソル {x.shape=}")

            # テキスト埋め込みをPatch-Mixerの次元に投影
            y_mixer = self.patch_mixer_map_y(y)
            logger.debug(f"Patch-Mixerテキスト埋め込みマッピング後のテンソル {y_mixer.shape=}")

            # Patch-Mixerブロックを順番に適用
            for block in self.patch_mixer:
                x = block(x, y_mixer, t)  # (N, T, D_mixer)
            logger.debug(f"Patch-Mixer適用後のテンソル {x.shape=}")

        # 3) マスキング
        
        if mask_ratio > 0:

            # パッチマスクを生成
            mask_dict = get_mask(
                x.shape[0], x.shape[1], 
                mask_ratio=mask_ratio, 
                device=x.device
            )
            ids_keep = mask_dict['ids_keep']
            ids_restore = mask_dict['ids_restore']
            mask = mask_dict['mask']

            # マスクアウトを適用
            x = mask_out_token(x, ids_keep)
            logger.debug(f"マスクアウト後のテンソル {x.shape=}")    
        
        if self.use_patch_mixer:
            # Patch-Mixerの出力をバックボーンのトランスフォーマー次元に投影
            # （計算を節約するためにマスキング後に実行）
            x = self.patch_mixer_map_xout(x)

        # 4) メインバックボーンの適用

        # DiTブロックを順番に適用
        for block in self.blocks:
            x = block(x, y, t)  # (N, T, D)

        # 5) アンマスキングと画像再構成

        # 最終出力層を適用
        # (N, T, patch_size ** 2 * out_channels)
        x = self.final_layer(x, t)
        logger.debug(f"最終出力層適用後のテンソル {x.shape=}")
        
        if mask_ratio > 0:
            # マスクトークンをゼロベクトルで補完し、元の順序に復元
            x = unmask_tokens(x, ids_restore, self.mask_token)

        # パッチを元の画像サイズに再構成
        # (N, out_channels, H, W)
        x = self.unpatchify(x)
        logger.debug(f"パッチ再構成後のテンソル {x.shape=}")

        return {'sample': x, 'mask': mask}
    
    def forward_with_cfg(
        self,
        x: torch.Tensor,
        t: torch.Tensor,
        y: torch.Tensor,
        cfg: float = 1.0,
        mask_ratio: float = 0,
        **kwargs
    ) -> dict:
        """
        Clissifier-Free Guidance（CFG）を使用した順伝播
        条件付き生成と、無条件での生成を同時に行い、結果を組み合わせる

        Args:
            x: 入力テンソル (batch_size, channels, height, width)
            t: タイムステップテンソル (batch_size,)
            y: キャプション埋め込みテンソル (batch_size, 1
            cfg: Classifier-free guidanceのスケール（1.0はガイダンスなし）
            mask_ratio: トレーニング中にマスクするパッチの割合（0から1の間）
        Returns:
            dict: 以下を含む辞書:
                - 'sample': 出力テンソル (batch_size, out_channels, height, width)
        """
        logger.info(f"DiTの順伝播（CFGあり） {x.shape=} {t.shape=} {y.shape=} {cfg=} {mask_ratio=}")

        # 1) バッチの拡張

        x = torch.cat([x, x], 0)
        logger.debug(f"CFG用にバッチを2倍に拡張 {x.shape=}")

        y = torch.cat([y, torch.zeros_like(y)], 0)
        logger.debug(f"CFG用にキャプション埋め込みを拡張 {y.shape=}")

        if len(t) != 1:
            t = torch.cat([t, t], 0)
            logger.debug(f"CFG用にタイムステップを拡張 {t.shape=}")
        
        # 2) 2倍のバッチサイズで順伝播を実行

        eps = self.forward_without_cfg(x, t, y, mask_ratio, **kwargs)['sample']

        # 3) 結果の分割とガイダンスの適用

        cond_eps, uncond_eps = torch.split(eps, len(eps) // 2, dim=0)
        logger.debug(f"条件付きと無条件の出力に分割 {cond_eps.shape=} {uncond_eps.shape=}")

        # CFGの式を適用
        # 無条件の出力 + スケール * (条件付きの出力 - 無条件の出力)
        eps = uncond_eps + cfg * (cond_eps - uncond_eps)
        logger.debug(f"CFGを適用した最終出力テンソル {eps.shape=}")

        return {'sample': eps}

    def forward(
        self,
        x: torch.Tensor,
        t: torch.Tensor,
        y: torch.Tensor,
        cfg: float = 1.0,
        **kwargs
    ) -> dict:
        """
        CFGの値に基づいて適切な順伝播にルーティングする

        Args:
            x: 入力テンソル (batch_size, channels, height, width)
            t: タイムステップテンソル (batch_size,)
            y: キャプション埋め込みテンソル (batch_size, 1, seq_len, dim)
            cfg: Classifier-free guidanceのスケール（1.0はガイダンスなし）
        Returns:
            dict: 以下を含む辞書:
                - 'sample': 出力テンソル (batch_size, out_channels, height, width)
        """

        # CFGを使用する場合
        if cfg != 1.0:
            return self.forward_with_cfg(x, t, y, cfg, **kwargs)

        # 使用しない場合
        else:
            return self.forward_without_cfg(x, t, y, **kwargs)

    def unpatchify(self, x: torch.Tensor) -> torch.Tensor:
        """
        パッチのシーケンスを元の画像の形状に復元する

        Args:
            x (torch.Tensor): パッチのシーケンス (N, num_patches, patch_size**2 * out_channels)
        Returns:
            torch.Tensor: 復元された画像テンソル (N, out_channels, H, W)
        """
        logger.info(f"パッチを元の画像に復元 {x.shape=}")

        # 1) 次元の計算と検証

        c = self.out_channels
        logger.debug(f"{c=}")

        p = self.x_embedder.patch_size[0]
        logger.debug(f"{p=}")

        h = w = int(x.shape[1]**0.5)
        logger.debug(f"{h=} {w=}")

        assert h * w == x.shape[1]

        # 2) フラットなパッチを画像に復元

        # (num_batch, num_patches, patch_size**2 * out_channels)
        # -> (num_batch, h, w, patch_size, patch_size, out_channels)
        x = x.reshape(shape=(x.shape[0], h, w, p, p, c))

        # 3) 次元の並び替え

        # (num_batch, out_channels, h*patch_size, w*patch_size)
        # -> (num_batch, out_channels, h, patch_size, w, patch_size)
        x = torch.einsum('nhwpqc->nchpwq', x)
        logger.debug(f"パッチを正しい位置に配置後のテンソル {x.shape=}")

        # 4) 画像への結合

        # (num_batch, out_channels, h, patch_size, w, patch_size)
        # -> (num_batch, out_channels, h*patch_size, h*patch_size)
        result = x.reshape(shape=(x.shape[0], c, h * p, h * p))
        logger.debug(f"復元された画像テンソル {result.shape=}")

        return result

    def initialize_weights(self) -> None:
        """
        各層のパラメータを初期化する
        """
        logger.info("重みの初期化を実行")

        # 1) 基本的な初期化を適用

        def zero_bias(m: nn.Module) -> None:
            """
            バイアスをゼロで初期化
            """
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

        def _basic_init(module: nn.Module) -> None:
            """
            Xavier（シビアー）初期化とバイアスのゼロ初期化を適用
            """
            if isinstance(module, nn.Linear):
                torch.nn.init.xavier_uniform_(module.weight)
                zero_bias(module)  # All bias in the model init to zero

        # Baseline init of all parameters
        self.apply(_basic_init)

        # 2) 位置埋め込みを初期化

        # 2次元正弦波位置埋め込みを計算
        pos_embed = get_2d_sincos_pos_embed(
            self.pos_embed.shape[-1],
            int(self.x_embedder.num_patches**0.5),
            pos_interp_scale=self.pos_interp_scale,
            base_size=self.base_size
        )

        # 学習可能な位置埋め込みに代入
        self.pos_embed.data.copy_(
            torch.from_numpy(pos_embed).float().unsqueeze(0)
        )

        # 3) 特定の層の正規分布初期化

        # Xavier初期化
        w = self.x_embedder.proj.weight.data
        nn.init.xavier_uniform_(w.view([w.shape[0], -1]))

        # 標準偏差0.02の正規分布で初期化
        nn.init.normal_(self.t_embedder.mlp[0].weight, std=0.02)
        nn.init.normal_(self.t_embedder.mlp[2].weight, std=0.02)
        nn.init.normal_(self.pooled_y_emb_process.fc1.weight, std=0.02)
        nn.init.normal_(self.pooled_y_emb_process.fc2.weight, std=0.02)
        nn.init.normal_(self.y_embedder.y_proj.fc1.weight, std=0.02)
        nn.init.normal_(self.y_embedder.y_proj.fc2.weight, std=0.02)
        
        # 4) ブロックごとのカスタム初期化

        for block in self.blocks:
            block.custom_init()

        for block in self.patch_mixer:
            block.custom_init()

        # AdaLNの変調パラメータをゼロで初期化
        for block in self.blocks:
            nn.init.constant_(block.adaLN_modulation[-1].weight, 0)

        # AdaLNの変調パラメータをゼロで初期化
        for block in self.patch_mixer:
            nn.init.constant_(block.adaLN_modulation[-1].weight, 0)

        self.y_emb_preprocess.custom_init()

        # テキスト前処理層の特定のパラメータをゼロで初期化
        nn.init.constant_(self.y_emb_preprocess.attn.proj.weight, 0)
        nn.init.constant_(self.y_emb_preprocess.mlp.w3.weight, 0)

        # 最終出力層をゼロで初期化
        nn.init.constant_(self.final_layer.adaLN_modulation[-1].weight, 0)
        nn.init.constant_(self.final_layer.linear.weight, 0)


In [ ]:
def MicroDiT_Tiny_2(
    caption_channels: int = 1024,
    qkv_ratio: List[float] = [0.5, 1.0],
    mlp_ratio: List[float] = [0.5, 4.0],
    pos_interp_scale: float = 1.0,
    input_size: int = 32,
    num_experts: int = 8,
    expert_capacity: float = 2.0,
    experts_every_n: int = 2,
    in_channels: int = 4,
    **kwargs
) -> DiT:
    depth = 16
    model = DiT(
        input_size=input_size,
        patch_size=2,
        in_channels=in_channels,
        dim=512,
        depth=depth,
        head_dim=32,
        multiple_of=256,
        caption_channels=caption_channels,
        pos_interp_scale=pos_interp_scale,
        norm_eps=1e-6,
        depth_init=True,
        qkv_multipliers=np.linspace(qkv_ratio[0], qkv_ratio[1], num=depth, dtype=float),
        ffn_multipliers=np.linspace(mlp_ratio[0], mlp_ratio[1], num=depth, dtype=float),
        use_patch_mixer=True,
        patch_mixer_depth=4,
        patch_mixer_dim=512,  # allocating higher budget to mixer layers
        patch_mixer_qkv_ratio=1.0,
        patch_mixer_mlp_ratio=4.0,
        use_bias=False,
        num_experts=num_experts,
        expert_capacity=expert_capacity,
        experts_every_n=experts_every_n,
        **kwargs
    )
    return model

In [ ]:
def MicroDiT_XL_2(
    caption_channels: int = 1024,
    qkv_ratio: List[float] = [0.5, 1.0],
    mlp_ratio: List[float] = [0.5, 4.0],
    pos_interp_scale: float = 1.0,
    input_size: int = 32,
    num_experts: int = 8,
    expert_capacity: float = 2.0,
    experts_every_n: int = 2,
    in_channels: int = 4,
    **kwargs
) -> DiT:
    depth = 28
    model = DiT(
        input_size=input_size,
        patch_size=2,
        in_channels=in_channels,
        dim=1024,
        depth=depth,
        head_dim=64,
        multiple_of=256,
        caption_channels=caption_channels,
        pos_interp_scale=pos_interp_scale,
        norm_eps=1e-6,
        depth_init=True,
        qkv_multipliers=np.linspace(qkv_ratio[0], qkv_ratio[1], num=depth, dtype=float),
        ffn_multipliers=np.linspace(mlp_ratio[0], mlp_ratio[1], num=depth, dtype=float),
        use_patch_mixer=True,
        patch_mixer_depth=6,
        patch_mixer_dim=768,  # allocating higher budget to mixer layers
        patch_mixer_qkv_ratio=1.0,
        patch_mixer_mlp_ratio=4.0,
        use_bias=False,
        num_experts=num_experts,
        expert_capacity=expert_capacity,
        experts_every_n=experts_every_n,
        **kwargs
    )
    return model

In [ ]:
micro_dit_tiny_2 = MicroDiT_Tiny_2()
sample_input = torch.randn(1, 4, 32, 32)
sample_timestep = torch.tensor([10])
sample_caption = torch.randn(1, 1, 16, 1024)
output = micro_dit_tiny_2(sample_input, sample_timestep, sample_caption, cfg=1.5, mask_ratio=0.3)
logger.info(f"MicroDiT_Tiny_2の出力テンソル {output['sample'].shape=}")

## LatentDiffusion

### UniversalTokenizer

In [ ]:
class simple_2_hf_tokenizer_wrapper:
    """
    OpenCLIPトークナイザーのラッパークラス
    Hugging Faceのトークナイザーに沿わせるために使用
    
    Args:
        tokenizer (Any): OpenCLIP tokenizer instance
    """
    def __init__(self, tokenizer: Any):
        """
        Args:
            tokenizer (Any): OpenCLIPのトークナイザーインスタンス
        """
        logger.info(f"OpenCLIPトークナイザーのラッパーを初期化")

        self.tokenizer = tokenizer

        # Hugging Faceのトークナイザーが持つ属性をモック
        self.model_max_length = self.tokenizer.context_length
        
    def __call__(
        self,
        caption: str,
        padding: str = 'max_length',
        max_length: Optional[int] = None,
        truncation: bool = True,
        **kwargs
    ) -> Dict[str, torch.Tensor]:
        logger.info(f"OpenCLIPでキャプションをトークン化: {caption}")

        input_ids = self.tokenizer(caption, context_length=max_length)
        logger.debug(f"トークン化結果: {input_ids}")

        return { 'input_ids': input_ids }

In [ ]:
def text_encoder_embedding_format(enc: str) -> Tuple[int, int]:
    """
    指定されたテキストエンコーダーの名前を受け取り、埋め込み表現の形状を返す
    テキスト条件づけで、入力されるテキスト特徴量のサイズを事前に知る必要があるため

    Args:
        enc (str): テキストエンコーダーの名前
    Returns:
        Tuple[int, int]:
            - トークンの最大長
            - トークンの埋め込み次元数
    """

    if enc in [
        'stabilityai/stable-diffusion-2-base',
        'runwayml/stable-diffusion-v1-5',
        'CompVis/stable-diffusion-v1-4'
    ]:
        return 77, 1024

    # OpenCLIPの場合
    if enc in ['openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378']:
        # トークンの最大長は77、埋め込み次元数は1024
        return 77, 1024

    if enc in ["DeepFloyd/t5-v1_1-xxl"]:
        return 120, 4096

    raise ValueError(f'Please specifcy the sequence and embedding size of {enc} encoder')

In [ ]:
class UniversalTokenizer:
    """
    OpenCLIPやT5などの異なる種類のトークナイザーに対応する汎用トークナイザークラス
    """

    def __init__(self, name: str):
        """
        Args:
            name (str): トークナイザーの名前またはモデル識別子
                - OpenCLIP
                - T5
                - CLIP
        """
        logger.info(f"UniversalTokenizerを初期化: {name=}")

        self.name = name

        s, d = text_encoder_embedding_format(name)

        # OpenCLIPの場合
        # True
        if self.name.startswith("openclip:"):

            # OpenCLIPトークナイザーのラッパークラスを初期化
            self.tokenizer = simple_2_hf_tokenizer_wrapper(
                open_clip.get_tokenizer(name.lstrip('openclip:'))
            )
            assert s == self.tokenizer.model_max_length, "simply check of text_encoder_embedding_format"

        # T5の場合
        elif self.name == "DeepFloyd/t5-v1_1-xxl":
            self.tokenizer = T5Tokenizer.from_pretrained(name) # for t5 we would use a smaller than max_seq_length

        # CLIPの場合
        else:
            self.tokenizer = CLIPTokenizer.from_pretrained(name, subfolder='tokenizer')
            assert s == self.tokenizer.model_max_length, "simply check of text_encoder_embedding_format"

        self.model_max_length = s
        
    def tokenize(self, captions: Union[str, List[str]]) -> Dict[str, torch.Tensor]:
        logger.info(f"キャプションをトークン化: {captions=}")

        # T5の場合
        # False
        if self.name == "DeepFloyd/t5-v1_1-xxl":
            text_tokens_and_mask = self.tokenizer(
                captions,
                padding='max_length',
                max_length=self.model_max_length,
                truncation=True,
                return_attention_mask=True,
                add_special_tokens=True,
                return_tensors='pt'
            )

            # パディングを無視するためのアテンションマスクが必要
            return {
                'input_ids': text_tokens_and_mask['input_ids'],
                'attention_mask': text_tokens_and_mask['attention_mask']
            }

        # OpenCLIPまたはCLIPの場合
        # True
        else:
            # アテンションマスクを明示的に使用しない
            tokenized_caption = self.tokenizer(
                captions,
                padding='max_length',
                max_length=self.tokenizer.model_max_length,
                truncation=True,
                return_tensors='pt'
            )['input_ids']
            return {'input_ids': tokenized_caption}

### UniversalTextEncoder

In [ ]:
class openclip_text_encoder(torch.nn.Module):
    """
    OpenCLIPテキストエンコーダーのラッパークラス
    入出力形式を使いやすくするために使用
    """
    def __init__(self, clip_model: Any, dtype: torch.dtype = torch.float32, **kwargs) -> None:
        """
        Args:
            clip_model (Any): OpenCLIPモデルインスタンス
            dtype (torch.dtype, torch.float32): モデル重みのデータ型
        """
        logger.info(f"OpenCLIPテキストエンコーダーのラッパーを初期化: {dtype=}")
        super().__init__()
        self.clip_model = clip_model
        self.device = None
        self.dtype = dtype

    def forward_fxn(self, text: torch.Tensor) -> Tuple[torch.Tensor, None]:
        """
        テキストエンコーダーの順伝搬

        Args:
            text (torch.Tensor): トークン化されたテキスト入力テンソル
        Returns:
            Tuple[torch.Tensor, None]: テキスト埋め込みテンソルとNone
        """
        logger.info(f"OpenCLIPテキストエンコーダーの順伝播（内部） {text.shape=}")

        # 1) テキストの埋め込み

        # キャストするデータ型を取得
        cast_dtype = self.clip_model.transformer.get_cast_dtype()
        logger.debug(f"キャストするデータ型を取得: {cast_dtype=}")

        # テキストの埋め込みを取得し、データ型をキャスト
        # (batch_size, n_ctx, d_model)
        x = self.clip_model.token_embedding(text).to(cast_dtype)
        logger.debug(f"テキストを埋め込みに変換: {x.shape=}")

        # 2) 位置エンコーディングの追加

        x = x + self.clip_model.positional_embedding.to(cast_dtype)


        # 3) Transformerを適用

        # (batch_size, n_ctx, d_model) -> (n_ctx, batch_size, d_model)
        x = x.permute(1, 0, 2)
        logger.debug(f"次元を入れ替え {x.shape=}")

        x = self.clip_model.transformer(x)

        # (n_ctx, batch_size, d_model) -> (batch_size, n_ctx, d_model)
        x = x.permute(1, 0, 2)
        logger.debug(f"次元を元に戻す {x.shape=}")

        # 4) レイヤー正規化を適用

        x = self.clip_model.ln_final(x)

        # 5) 出力を整形（DiTの入力やHugging Faceの仕様に合わせる）

        # (batch_size, n_ctx, d_model) -> (batch_size, 1, n_ctx, d_model)
        x = x.unsqueeze(dim=1)

        return x, None

    def forward(self, text: torch.Tensor, **kwargs) -> Tuple[torch.Tensor, None]:
        logger.info(f"OpenCLIPテキストエンコーダーの順伝播 {text.shape=}")

        # CUDAで指定されたデータ型を使って内部の順伝播を実行
        with torch.autocast(device_type='cuda', dtype=self.dtype):
            return self.forward_fxn(text)

In [ ]:
class UniversalTextEncoder(torch.nn.Module):
    """
    異なるテキストエンコーダーに対応する汎用テキストエンコーダークラス
    
    Args:
        name (str): Name/path of the model to load
        dtype (str): Data type for model weights
        pretrained (bool, True): Whether to load pretrained weights
    """

    def __init__(self, name: str, dtype: str, pretrained: bool = True):
        """
        Args:
            name (str): モデルの名前またはパス
            dtype (str): モデル重みのデータ型
            pretrained (bool, True): 事前学習済み重みをロードするかどうか
        """
        logger.info(f"UniversalTextEncoderを初期化: {name=}, {dtype=}, {pretrained=}")

        super().__init__()

        self.name = name

        # OpenCLIPの場合
        # True
        if self.name.startswith("openclip:"):
            assert pretrained, 'Load default pretrained model from openclip'

            # OpenCLIPテキストエンコーダーのラッパークラスを初期化
            self.encoder = openclip_text_encoder(
                open_clip.create_model_and_transforms(name.lstrip('openclip:'))[0],
                torch_dtype=DATA_TYPES[dtype]
            )

        # False
        elif self.name == "DeepFloyd/t5-v1_1-xxl":
            self.encoder = T5EncoderModel.from_pretrained(
                name,
                torch_dtype=DATA_TYPES[dtype],
                pretrained=pretrained
            )

        # False
        else:
            self.encoder = CLIPTextModel.from_pretrained(
                name,
                subfolder='text_encoder',
                torch_dtype=DATA_TYPES[dtype],
                pretrained=pretrained
            )

    def encode(self, tokenized_caption: torch.Tensor, attention_mask: Optional[torch.Tensor] = None) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        """
        テキストキャプションをエンコードして埋め込みを取得
        Args:
            tokenized_caption (torch.Tensor):
                トークン化されたキャプションテンソル
            attention_mask (Optional[torch.Tensor], None):
                アテンションマスクテンソル（T5用）
        Returns:
            Tuple[torch.Tensor, Optional[torch.Tensor]]:
                埋め込みテンソルとオプションのアテンションマスク
        """
        logger.info(f"UniversalTextEncoderでキャプションをエンコード: {tokenized_caption.shape=}, {attention_mask.shape if attention_mask is not None else None=}")

        # T5の場合
        # False
        if self.name == "DeepFloyd/t5-v1_1-xxl":
            out = self.encoder(
                tokenized_caption,
                attention_mask=attention_mask
            )['last_hidden_state']
            out = out.unsqueeze(dim=1)
            return out, None

        # True
        else:
            # OpenCLIPテキストエンコーダーで順伝播
            return self.encoder(tokenized_caption)


### DistLoss

In [ ]:
class DistLoss(Metric):
    """
    分散学習環境で損失の平均値を集計するクラス
    torchmetrics.Metricを継承
    """
    def __init__(self, **kwargs):
        """
        Args:
            kwargs (Any): 親クラスに渡される追加の引数
        """
        logger.info("DistLossメトリックを初期化")

        super().__init__(**kwargs)

        # 損失を集計するための状態を追加
        self.add_state(
            "loss", default=torch.tensor(0.), dist_reduce_fx="sum"
        )

        # バッチ数を集計するための状態を追加
        self.add_state(
            "batches", default=torch.tensor(0), dist_reduce_fx="sum"
        )

    def update(self, value: torch.Tensor) -> None:
        """
        メトリックの状態を更新する

        Args:
            value (torch.Tensor): 現在のバッチの損失値
        """
        logger.info(f"DistLossメトリックを更新: {value=}")

        # 損失を加算
        self.loss += value

        # バッチ数をインクリメント
        self.batches += 1

    def compute(self) -> torch.Tensor:
        """
        集計された損失の平均値を計算する

        Returns:
            torch.Tensor: 集計された損失の平均値
        """
        return self.loss.float() / self.batches

### LatentDiffusion

In [ ]:
class LatentDiffusion(ComposerModel):
    """
    テキストプロンプトから画像を生成するLatent Diffusionモデル（LDM）

    - MosaicMLのComposerModelを継承し、Composerという学習フレームワークと統合
    - 3つの主要コンポーネント
        - DiT: ノイズ除去を行うメインのモデル（学習対象）
        - VAE（AutoencoderKL）: 画像を潜在空間に圧縮・復元するモデル（凍結）
        - テキストエンコーダー（UniversalTextEncoder）:
            テキストを埋め込みに変換するモデル（凍結）
    - EDM（Elucidated Diffusion Model）サンプリングプロセスを実装
        - EDMは、ノイズスケジュールとサンプリング手順のベストプラクティス
    - 画像とテキストの埋め込みは、事前計算されたものを使用可能
    """

    def __init__(
        self,
        dit: nn.Module,
        vae: AutoencoderKL,
        text_encoder: UniversalTextEncoder,
        tokenizer: UniversalTokenizer,
        image_key: str = 'image',
        text_key: str = 'captions',
        image_latents_key: str = 'image_latents',
        text_latents_key: str = 'caption_latents',
        precomputed_latents: bool = True,
        dtype: str = 'bfloat16',
        latent_res: int = 32,
        p_mean: float = -0.6,
        p_std: float = 1.2,
        train_mask_ratio: float = 0.
    ):
        """
        Args:
            dit (nn.Module): 学習対象のDiffusion Transformerモデル
            vae (AutoencoderKL): 画像を潜在空間にエンコード/デコードするVAEモデル（凍結）
            text_encoder (UniversalTextEncoder): テキストプロンプトを埋め込みに変換するテキストエンコーダー（凍結）
            tokenizer (UniversalTokenizer): テキストプロンプトを処理するトークナイザー
            image_key (str, optional): バッチ辞書内の画像データのキー。デフォルトは 'image'。
            text_key (str, optional): バッチ辞書内のテキストデータのキー。 デフォルトは 'captions'。
            image_latents_key (str, optional): バッチ辞書内の事前計算された画像潜在変数のキー。デフォルトは 'image_latents'。
            text_latents_key (str, optional): バッチ辞書内の事前計算されたテキスト潜在変数のキー。デフォルトは 'caption_latents'。
            precomputed_latents (bool, optional): 事前計算された潜在変数を使用するかどうか。デフォルトは True。
            dtype (str, optional): 計算精度。デフォルトは 'bfloat16'。
            latent_res (int, optional): VAEによる8倍のダウンサンプリングを想定した潜在空間の解像度。デフォルトは32。
            p_mean (float, optional): EDMの対数正規ノイズの平均。デフォルトは -0.6。
            p_std (float, optional): EDMの対数正規ノイズの標準偏差。デフォルトは1.2。
            train_mask_ratio (float, optional): 学習中に画像をマスクする割合。デフォルトは0。
        """

        super().__init__()
        logger.info(f"LatentDiffusionモデルを初期化: {dtype=}, {latent_res=}, {precomputed_latents=}, {train_mask_ratio=}")

        # 1) 属性の設定

        self.dit = dit
        self.vae = vae
        self.image_key = image_key
        self.text_key = text_key
        self.image_latents_key = image_latents_key
        self.text_latents_key = text_latents_key
        self.precomputed_latents = precomputed_latents
        self.dtype = dtype
        self.latent_res = latent_res

        # EDMフレームに基づいたパラメータの設定
        self.edm_config = EasyDict({
            'sigma_min': 0.002, # 最小ノイズレベル
            'sigma_max': 80, # 最大ノイズレベル
            'P_mean': p_mean, # ノイズの対数正規分布の平均
            'P_std': p_std, # ノイズの対数正規分布の標準偏差
            'sigma_data': 0.9, # データの標準偏差
            'num_steps': 18, # サンプリングステップ数
            'rho': 7, # サンプリングのrhoパラメータ
            'S_churn': 0, # サンプリングのchurnパラメータ
            'S_min': 0, # サンプリングの最小値
            'S_max': float('inf'), # サンプリングの最大値
            'S_noise': 1 # サンプリングのノイズレベル
        })

        self.train_mask_ratio = train_mask_ratio

        # 評価・生成時はマスクなし
        self.eval_mask_ratio = 0.

        assert self.train_mask_ratio >= 0, 'Masking ratio must be non-negative!'

        self.randn_like = torch.randn_like

        # VAEのスケーリング係数を取得
        # 潜在空間が標準正規分布からずれていることがあるため
        self.latent_scale = self.vae.config.scaling_factor

        self.text_encoder = text_encoder
        self.tokenizer = tokenizer

        # 2) モデルの凍結

        # テキストエンコーダーを凍結
        self.text_encoder.requires_grad_(False)

        # VAEを凍結
        self.vae.requires_grad_(False)

        # FSDP（Fully Sharded Data Parallel）でラップしないように設定
        # FSDPは分散学習のための技術
        self.text_encoder._fsdp_wrap = False
        self.vae._fsdp_wrap = False
        self.dit._fsdp_wrap = True

    def forward(self, batch: dict) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        生データもしくは事前計算データを順伝搬し、損失を計算する

        Args:
            batch (dict): バッチデータの辞書
        Returns:
            tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
                - 計算された損失テンソル
                - 画像潜在変数テンソル
                - テキスト埋め込みテンソル
        """

        # 1) 画像の潜在変数の取得

        # 事前計算済みの場合
        if self.precomputed_latents and self.image_latents_key in batch:

            # そのまま使用
            latents = batch[self.image_latents_key]
            logger.debug(f"事前計算された画像潜在変数を使用 {latents.shape=}")

        # エンコードが必要な場合
        else:
            with torch.no_grad():
                # 画像を取得
                images = batch[self.image_key]

                # 画像を潜在空間にエンコード
                latents = self.vae.encode(
                    images.to(DATA_TYPES[self.dtype])
                )['latent_dist'].sample().data

                # VAEのスケーリング係数を乗じる
                latents *= self.latent_scale

            logger.debug(f"エンコードされた画像潜在変数 {latents.shape=}")

        # 2) テキスト埋め込みの取得

        # 事前計算済みの場合
        if self.precomputed_latents and self.text_latents_key in batch:

            # そのまま使用
            conditioning = batch[self.text_latents_key]

        # エンコードが必要な場合
        else:

            # キャプションを取得
            captions = batch[self.text_key]
            logger.debug(f"キャプションを取得 {captions=}")

            # キャプションを平坦化
            captions = captions.view(-1, captions.shape[-1])
            logger.debug(f"キャプションを平坦化 {captions.shape=}")

            # アテンションマスクがある場合（T5用）
            if 'attention_mask' in batch:
                conditioning = self.text_encoder.encode(
                    captions,
                    attention_mask=batch['attention_mask'].view(-1, captions.shape[-1])
                )[0]

            # ない場合（OpenCLIPやCLIP用）
            else:
                conditioning = self.text_encoder.encode(captions)[0]
                logger.debug(f"テキスト埋め込みを取得 {conditioning.shape=}") 

        # drop_caption_maskがある場合（CFG用）
        if 'drop_caption_mask' in batch.keys():

            # 一定確率でテキスト埋め込みをゼロにする
            conditioning *= batch['drop_caption_mask'].view(
                [-1] + [1] * (len(conditioning.shape) - 1)
            )

        # 3) EDM損失の計算

        # DiTモデルの順伝播を計算し、EDM損失を計算
        loss = self.edm_loss(
            latents.float(),
            conditioning.float(),
            mask_ratio=self.train_mask_ratio if self.training else self.eval_mask_ratio
        )
        return (loss, latents, conditioning)

    def model_forward_wrapper(
        self,
        x: torch.Tensor,
        sigma: torch.Tensor,
        y: torch.Tensor,
        model_forward_fxn: callable,
        mask_ratio: float,
        **kwargs
    ) -> dict:
        """
        EDM(Elucidated Diffusion Models)における事前条件付けするラッパー
        https://github.com/NVlabs/edm/blob/main/training/networks.py#L632

        ニューラルネットワークが学習しやすい範囲にスケーリングする（分散1付近）

        Args:
            x (torch.Tensor): 入力テンソル
            sigma (torch.Tensor): ノイズレベルテンソル
            y (torch.Tensor): 条件付け埋め込みテンソル
            model_forward_fxn (callable): DiTモデルの順伝播関数
            mask_ratio (float): マスク比率
            **kwargs: 追加のキーワード引数

        Returns:
            dict: モデルの出力辞書
        """
        logger.info(f"EDMモデル順伝播ラッパーを呼び出し: {x.shape=}, {sigma.shape=}, {y.shape=}, {mask_ratio=}")

        # ノイズレベルを適切な形状に変換
        sigma = sigma.to(x.dtype).reshape(-1, 1, 1, 1)
        logger.debug(f"ノイズレベルを変換 {sigma.shape=}")

        # 入力画像をそのまま出力に足し合わせる割合（スキップ接続）
        c_skip = (
            self.edm_config.sigma_data ** 2 /
            (sigma ** 2 + self.edm_config.sigma_data ** 2)
        )

        # モデルの出力をスケーリングする係数
        c_out = (
            sigma * self.edm_config.sigma_data /
            (sigma ** 2 + self.edm_config.sigma_data ** 2).sqrt()
        )

        # モデルに入力する画像をスケーリングする係数
        c_in = 1 / (self.edm_config.sigma_data ** 2 + sigma ** 2).sqrt()

        # モデルに現在のノイズの強さを伝えるための値
        c_noise = sigma.log() / 4

        # 2) モデルの実行

        # DiTモデルの順伝播を呼び出す
        out = model_forward_fxn(
            (c_in * x).to(x.dtype),
            c_noise.flatten(),
            y,
            mask_ratio=mask_ratio,
            **kwargs
        )

        # モデルの生の出力を取得
        F_x = out['sample']

        # 3) 最終的な予測値の合成

        # すべてのテンソルを同じデバイスに移動
        c_skip = c_skip.to(F_x.device)
        x = x.to(F_x.device)
        c_out = c_out.to(F_x.device)

        # 元の入力とモデルの出力を合成して最終的な予測を得る
        D_x = c_skip * x + c_out * F_x

        out['sample'] = D_x
        return out

    def edm_loss(self, x: torch.Tensor, y: torch.Tensor, mask_ratio: float = 0, **kwargs) -> torch.Tensor:
        """
        EDM(Elucidated Diffusion Models)損失計算

        Args:
            x (torch.Tensor): クリーンな画像テンソル
            y (torch.Tensor): 条件付け埋め込みテンソル
            mask_ratio (float, optional): マスク比率。デフォルトは0。
        Returns:
            torch.Tensor: 計算された損失テンソル
        """
        logger.info(f"EDM損失を計算: {x.shape=}, {y.shape=}, {mask_ratio=}")

        # 1) ノイズレベルのサンプリング

        # 標準正規分布から乱数を生成
        rnd_normal = torch.randn([x.shape[0], 1, 1, 1], device=x.device)

        # 対数正規分布からノイズレベルをサンプリング
        sigma = (rnd_normal * self.edm_config.P_std + self.edm_config.P_mean).exp()

        # 2) 損失の重み付けの計算

        # ノイズレベルに応じて、損失関数の重みを計算
        # ノイズが大きいほど、損失の重みが小さくなる
        weight = (
            (sigma ** 2 + self.edm_config.sigma_data ** 2) /
            (sigma * self.edm_config.sigma_data) ** 2
        )

        # 3) ノイズの追加とモデルの順伝播

        # ノイズを生成
        n = self.randn_like(x) * sigma

        #ノイズを追加し、モデルの順伝播を実行
        model_out = self.model_forward_wrapper(
            x + n, # ノイズを追加
            sigma,
            y,
            self.dit,
            mask_ratio=mask_ratio,
            **kwargs
        )

        # ノイズが除去された画像を取得
        D_xn = model_out['sample']

        # 4) 基本的な損失の計算

        # 予測画像と正解画像の二乗誤差を計算し、重みを適用
        loss = weight * ((D_xn - x) ** 2)  # (N, C, H, W)
        logger.debug(f"基本的な損失を計算 {loss.shape=}")

        # 5) マスクがある場合の損失計算

        # マスクがある場合
        if mask_ratio > 0:
            # Masking is not feasible during image generation as it only returns denoised version
            # for non-masked patches. Image generation requires all patches to be denoised.
            assert (
                self.dit.training and 'mask' in model_out
            ), 'Masking is only recommended during training'
            loss = F.avg_pool2d(loss.mean(dim=1), self.dit.patch_size).flatten(1)

            # マスクされていない部分を取得
            unmask = 1 - model_out['mask']
            logger.debug(f"マスクされていない部分を取得 {unmask.shape=}")

            # マスクされていない部分の損失のみを計算
            loss = (loss * unmask).sum(dim=1) / unmask.sum(dim=1)  # (N,)
            logger.debug(f"マスクされていない部分の損失を計算 {loss.shape=}")

        # 平均損失を返す
        return loss.mean()

    def loss(self, outputs: tuple, batch: dict) -> torch.Tensor:
        """
        Composer特有の損失関数ラッパー

        Args:
            outputs (tuple): モデルの出力タプル
            batch (dict): バッチデータの辞書
        Returns:
            torch.Tensor: 計算された損失テンソル
        """
        logger.info("loss関数ラッパーを呼び出し")

        # forwardメソッドで計算済みの損失をそのまま返す
        return outputs[0]

    def eval_forward(self, batch: dict, outputs: Optional[tuple] = None) -> tuple:
        """
        Composer特有の評価用順伝播ラッパー

        Args:
            batch (dict): バッチデータの辞書
            outputs (Optional[tuple], None): 事前計算された出力タプル
        Returns:
            tuple: 評価用の出力タプル
        """
        logger.info("eval_forwardを呼び出し")

        # 計算済みの場合
        if outputs is not None:
            # そのまま返す
            return outputs

        # 計算されていない場合はforwardメソッドを呼び出し損失を計算
        loss, _, _ = self.forward(batch)

        return loss, None, None

    def get_metrics(self, is_train: bool = False) -> dict:
        """
        Composer特有のメトリック取得ラッパー

        Args:
            is_train (bool, optional): 訓練モードかどうか。デフォルトはFalse。
        Returns:
            dict: メトリック辞書
        """
        logger.info(f"get_metricsを呼び出し {is_train=}")
        return {'loss': DistLoss()}

    def update_metric(self, batch: dict, outputs: tuple, metric: DistLoss):
        """
        Composer特有のメトリック更新ラッパー

        Args:
            batch (dict): バッチデータの辞書
            outputs (tuple): モデルの出力タプル
            metric (DistLoss): 更新するメトリックインスタンス
        """
        logger.info("update_metricを呼び出し")
        metric.update(outputs[0])

    @torch.no_grad()
    def edm_sampler_loop(
        self,
        x: torch.Tensor, 
        y: torch.Tensor, 
        steps: Optional[int] = None, 
        cfg: float = 1.0, 
        **kwargs
    ) -> torch.Tensor:
        """
        EDM(Elucidated Diffusion Models)に基づいて画像を生成するサンプリングループ
        Args:
            x (torch.Tensor): 初期ノイズテンソル
            y (torch.Tensor): 条件付け埋め込みテンソル
            steps (Optional[int], None): サンプリングステップ数。Noneの場合はデフォルトを使用。デフォルトはNone。
            cfg (float, optional): Classifier-Free Guidanceのスケール。デフォルトは1.0。
            **kwargs: 追加のキーワード引数
        Returns:
            torch.Tensor: 生成された画像テンソル
        """

        # 1) サンプリングループの初期化

        # マスクの割合はゼロ
        mask_ratio = 0

        # モデルの順伝播関数を設定（CFGが有効な場合は部分適用）
        model_forward_fxn = (
            partial(self.dit.forward, cfg=cfg) if cfg > 1.0
            else self.dit.forward
        )

        # 2) サンプリングスケジュールの設定

        # タイムステップ数
        num_steps = self.edm_config.num_steps if steps is None else steps
        logger.debug(f"サンプリングステップ数を設定 {num_steps=}")

        # インデックス配列を作成
        # [0, 1, 2, ..., num_steps-1]
        step_indices = torch.arange(num_steps, dtype=torch.float64, device=x.device)

        # EDMフレームワークに基づいて、タイムステップごとのノイズレベルを計算
        t_steps = (
            self.edm_config.sigma_max ** (1 / self.edm_config.rho) +
            step_indices / (num_steps - 1) *
            (self.edm_config.sigma_min ** (1 / self.edm_config.rho) -
             self.edm_config.sigma_max ** (1 / self.edm_config.rho))
        ) ** self.edm_config.rho

        # 最後のステップにゼロを追加
        t_steps = torch.cat([torch.as_tensor(t_steps), torch.zeros_like(t_steps[:1])])

        # 3) サンプリングループの実行

        # 初期ノイズを、最初のノイズレベルでスケーリング
        x_next = x.to(torch.float64) * t_steps[0]

        # タイムステップごとにループ
        # t_cur: 現在のタイムステップ　t_next: 次のタイムステップ
        for i, (t_cur, t_next) in enumerate(zip(t_steps[:-1], t_steps[1:])):  # 0, ..., N-1

            # 現在の画像を設定
            x_cur = x_next

            # 確率的撹乱（Stochastic Churn)
            # ノイズを一時的に増加させる
            # 決定論的なODEサンプリングに、確率的な要素（SDE）を混ぜ、画質を向上させる
            gamma = (
                min(self.edm_config.S_churn / num_steps, np.sqrt(2) - 1)
                if self.edm_config.S_min <= t_cur <= self.edm_config.S_max else 0
            )

            # ノイズを増加させた新しいタイムステップ
            t_hat = torch.as_tensor(t_cur + gamma * t_cur)

            # 実際に画像にノイズを加える
            x_hat = (
                x_cur +
                (t_hat ** 2 - t_cur ** 2).sqrt() *
                self.edm_config.S_noise *
                self.randn_like(x_cur)
            )

            # 3) オイラー法による予測と移動

            # DiTを使って、ノイズ除去された画像を予測し、倍精度にアップキャスト
            denoised = self.model_forward_wrapper(
                x_hat.to(torch.float32),
                t_hat.to(torch.float32),
                y,
                model_forward_fxn,
                mask_ratio=mask_ratio,
                **kwargs
            )['sample'].to(torch.float64)

            # 現在位置から綺麗な画像への勾配を計算
            d_cur = (x_hat - denoised) / t_hat

            # 次のタイムステップ(t_next)へ向かって移動
            x_next = x_hat + (t_next - t_hat) * d_cur

            # 2) ホイン法（Heun's method）による精度向上

            if i < num_steps - 1:

                # オイラー法で到達した仮の位置で再度予測し、倍精度にアップキャスト
                denoised = self.model_forward_wrapper(
                    x_next.to(torch.float32),
                    t_next.to(torch.float32),
                    y,
                    model_forward_fxn,
                    mask_ratio=mask_ratio,
                    **kwargs
                )['sample'].to(torch.float64)

                # 仮の地点から綺麗な画像への勾配を計算
                d_prime = (x_next - denoised) / t_next

                # 最初の傾きと、オイラー法で到達した傾きを平均化して、移動位置を決定
                x_next = x_hat + (t_next - t_hat) * (0.5 * d_cur + 0.5 * d_prime)

        return x_next.to(torch.float32)

    @torch.no_grad()
    def generate(
        self,
        prompt: Optional[list] = None,
        tokenized_prompts: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.LongTensor] = None,
        guidance_scale: Optional[float] = 5.0,
        num_inference_steps: Optional[int] = 30,
        seed: Optional[int] = None,
        return_only_latents: Optional[bool] = False,
        **kwargs
    ) -> torch.Tensor:
        """
        学習済みのモデルを使用して、テキストプロンプトから画像を生成する

        Args:
            prompt (Optional[list], None): テキストプロンプトのリスト。デフォルトはNone。
            tokenized_prompts (Optional[torch.LongTensor], None): トークン化されたプロンプトテンソル。デフォルトはNone。
            attention_mask (Optional[torch.LongTensor], None): アテンションマスクテンソル（T5用）。デフォルトはNone。
            guidance_scale (Optional[float], 5.0): Classifier-Free Guidanceのスケール。デフォルトは5.0。
            num_inference_steps (Optional[int], 30): サンプリングステップ数。デフォルトは30。
            seed (Optional[int], None): 乱数シード。デフォルトはNone。
            return_only_latents (Optional[bool], False): 潜在変数のみを返すかどうか。デフォルトはFalse。
            **kwargs: 追加のキーワード引数
        Returns:
            torch.Tensor: 生成された画像テンソルもしくは潜在変数テンソル
        """
        logger.info(f"画像を生成: {prompt=}, {tokenized_prompts is not None=}, {guidance_scale=}, {num_inference_steps=}, {seed=}, {return_only_latents=}")

        # 1) 準備

        # _check_prompt_given(prompt, tokenized_prompts, prompt_embeds=None)
        assert prompt or tokenized_prompts, "Must provide either prompt or tokenized prompts"

        # モデルのデバイスを取得
        device = self.vae.device

        # 乱数生成器を初期化
        rng_generator = torch.Generator(device=device)

        # シードが指定されている場合
        if seed:
            # 乱数生成器にシードを設定
            rng_generator = rng_generator.manual_seed(seed)

        # 2) テキスト埋め込みの取得

        # トークン化されていない場合
        if tokenized_prompts is None:

            # トークナイザーでプロンプトをトークン化
            out = self.tokenizer.tokenize(prompt)

            # トークンIDのシーケンスを取得
            tokenized_prompts = out['input_ids']
            logger.debug(f"プロンプトをトークン化 {tokenized_prompts.shape=}")

            # T5用のアテンションマスクを取得
            attention_mask = (
                out['attention_mask'] if 'attention_mask' in out else None
            )

        # トークン化されたプロンプトをテキストエンコーダーでエンコード
        text_embeddings = self.text_encoder.encode(
            tokenized_prompts.to(device),
            attention_mask=attention_mask.to(device) if attention_mask is not None else None
        )[0]
        logger.debug(f"テキスト埋め込みを取得 {text_embeddings.shape=}")

        # 3) 初期ノイズを生成

        # ガウス分布からランダムなノイズを作成（潜在変数）
        latents = torch.randn(
            (len(text_embeddings), self.dit.in_channels, self.latent_res, self.latent_res),
            device=device,
            generator=rng_generator,
        )
        logger.debug(f"初期ノイズを生成 {latents.shape=}")

        # 4) EDMサンプリングループで画像を生成

        # ノイズ除去のサンプリングループを実行
        latents = self.edm_sampler_loop(
            latents,
            text_embeddings,
            num_inference_steps,
            cfg=guidance_scale
        )

        # 潜在変数のみを返す場合
        if return_only_latents:
            return latents

        # 5) 画像をデコードして返す

        # スケールを戻す
        latents = 1 / self.latent_scale * latents

        # データ型を取得
        torch_dtype = DATA_TYPES[self.dtype]
        logger.debug(f"デコード用のデータ型を取得 {torch_dtype=}")

        # VAEで潜在変数をデコードして画像に変換
        image = self.vae.decode(latents.to(torch_dtype)).sample

        # 画像のピクセル値を[0, 1]の範囲にクリップ
        image = (image / 2 + 0.5).clamp(0, 1)

        # 浮動小数点型に変換して返す
        image = image.float().detach()

        return image

In [ ]:
def create_latent_diffusion(
    vae_name: str = 'stabilityai/stable-diffusion-xl-base-1.0',
    text_encoder_name: str = 'openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378', 
    dit_arch: str = 'MicroDiT_XL_2',
    latent_res: int = 32,
    in_channels: int = 4,
    pos_interp_scale: float = 1.0,
    dtype: str = 'bfloat16',
    precomputed_latents: bool = True,
    p_mean: float = -0.6,
    p_std: float = 1.2,
    train_mask_ratio: float = 0.
) -> LatentDiffusion:
    """
    Latent Diffusionモデル（LDM）を作成するユーティリティ関数

    Args:
        vae_name (str, optional): 使用するVAEモデルの名前。デフォルトは 'stabilityai/stable-diffusion-xl-base-1.0'。
        text_encoder_name (str, optional): 使用するテキストエンコーダーの名前。デフォルトは 'openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378'。
        dit_arch (str, optional): 使用するDiTアーキテクチャの名前。デフォルトは 'MicroDiT_XL_2'。
        latent_res (int, optional): 潜在空間の解像度。デフォルトは32。
        in_channels (int, optional): DiTモデルの入力チャネル数。デフォルトは4。
        pos_interp_scale (float, optional): 位置埋め込みの補間スケール。デフォルトは1.0。
        dtype (str, optional): 計算精度。デフォルトは 'bfloat16'。
        precomputed_latents (bool, optional): 事前計算された潜在変数を使用するかどうか。デフォルトはTrue。
        p_mean (float, optional): EDMの対数正規ノイズの平均。デフォルトは -0.6。
        p_std (float, optional): EDMの対数正規ノイズの標準偏差。デフォルトは1.2。
        train_mask_ratio (float, optional): 学習中に画像をマスクする割合。デフォルトは0。
    Returns:
        LatentDiffusion: 作成されたLatent Diffusionモデルのインスタンス
    """
    logger.info(f"Latent Diffusionモデルを作成: {vae_name=}, {text_encoder_name=}, {dit_arch=}, {latent_res=}, {in_channels=}, {pos_interp_scale=}, {dtype=}, {precomputed_latents=}, {p_mean=}, {p_std=}, {train_mask_ratio=}")

    # 1) テキストエンコーダーの情報を抽出

    # 最大シーケンス長(s)とトークン埋め込み次元(d)を取得
    s, d = text_encoder_embedding_format(text_encoder_name)

    # 2) DiTの初期化

    # Tinyの場合
    if dit_arch == 'MicroDiT_Tiny_2':
        dit = MicroDiT_Tiny_2(
            input_size=latent_res,
            caption_channels=d,
            pos_interp_scale=pos_interp_scale,
            in_channels=in_channels
        )

    # XLの場合
    elif dit_arch == 'MicroDiT_XL_2':
        dit = MicroDiT_XL_2(
            input_size=latent_res,
            caption_channels=d,
            pos_interp_scale=pos_interp_scale,
            in_channels=in_channels
        )
    # その他
    else:
        raise ValueError(f'Unknown DiT architecture: {dit_arch}')

    # 3) VAEの初期化

    vae = AutoencoderKL.from_pretrained(
        vae_name,
        subfolder=None if vae_name=='ostris/vae-kl-f8-d16' else 'vae',
        torch_dtype=DATA_TYPES[dtype],
        pretrained=True
    )

    # 4) テキストエンコーダーの準備

    text_encoder = UniversalTextEncoder(
        text_encoder_name,
        dtype=dtype,
        pretrained=True
    )

    # 5) トークナイザーの準備

    tokenizer = UniversalTokenizer(text_encoder_name)

    # 6) Latent Diffusionモデルの作成

    model = LatentDiffusion(
        dit=dit,
        vae=vae,
        text_encoder=text_encoder,
        tokenizer=tokenizer,
        precomputed_latents=precomputed_latents,
        dtype=dtype,
        latent_res=latent_res,
        p_mean=p_mean,
        p_std=p_std,
        train_mask_ratio=train_mask_ratio
    )

    return model

In [ ]:
model = create_latent_diffusion(
    vae_name='stabilityai/stable-diffusion-xl-base-1.0',
    text_encoder_name='openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378',
    # dit_arch='MicroDiT_XL_2',
    dit_arch='MicroDiT_Tiny_2',
    latent_res=32,
    in_channels=4,
    pos_interp_scale=1.0,
    dtype='bfloat16',
    precomputed_latents=True,
    p_mean=-0.6,
    p_std=1.2,
    train_mask_ratio=0.75
)

## データセット構築

### ダウンロード

In [ ]:
def download_and_process_metadata():
    # Only using a single process for downloading metadata
    metadata_files = [
        ('data/train', 'train_anno.jsonl.tgz'),
        ('data/train', 'train_anno_realease_repath.jsonl.tgz'),
        ('data/valid', 'valid_anno_repath.jsonl.tgz'),
        ('data/test', 'test_questions.jsonl.tgz'),
        ('data/test', 'imgs.tgz'),
    ]

    for subfolder, filename in metadata_files:
        hf_hub_download(
            repo_id="JourneyDB/JourneyDB",
            repo_type="dataset",
            subfolder=subfolder,
            filename=filename,
            local_dir=COMPRESSED_DIR,
            local_dir_use_symlinks=False,
        )

    metadata_tars = [
        os.path.join(dir, fname) for (dir, fname) in metadata_files
    ]

    for tar_file in metadata_tars:
        subprocess.call(
            f'tar -xvzf {os.path.join(COMPRESSED_DIR, tar_file)} '
            f'-C {os.path.join(COMPRESSED_DIR, os.path.dirname(tar_file))}',
            shell=True,
        )

    shutil.copy(
        f'{os.path.join(COMPRESSED_DIR, "data/train/train_anno_realease_repath.jsonl")}',
        f'{os.path.join(RAW_DIR, "train/train_anno_realease_repath.jsonl")}',
    )

    shutil.copy(
        f'{os.path.join(COMPRESSED_DIR, "data/valid/valid_anno_repath.jsonl")}',
        f'{os.path.join(RAW_DIR, "valid/valid_anno_repath.jsonl")}',
    )

    shutil.copy(
        f'{os.path.join(COMPRESSED_DIR, "data/test/test_questions.jsonl")}',
        f'{os.path.join(RAW_DIR, "test/test_questions.jsonl")}',
    )

    shutil.move(
        f'{os.path.join(COMPRESSED_DIR, "data/test/imgs")}',
        f'{os.path.join(RAW_DIR, "test/")}',
    )

COMPRESSED_DIR = os.path.join(DATA_DIR, 'compressed')
logger.info(f"{COMPRESSED_DIR=}")

RAW_DIR = os.path.join(DATA_DIR, 'raw')
logger.info(f"{RAW_DIR=}")

if not os.path.exists(COMPRESSED_DIR):
    os.makedirs(COMPRESSED_DIR, exist_ok=True)
    os.makedirs(RAW_DIR, exist_ok=True)
    download_and_process_metadata()

### 解凍とリサイズ

In [ ]:
def download_uncompress_resize(
    valid_ids: list,
    max_image_size: int,
    min_image_size: int,
    split: str,
    idx: int,
):
    """Download, uncompress, and resize images for a given archive index."""
    assert split in ('train', 'valid')
    assert idx in valid_ids

    print(f"Downloading idx: {idx}")
    if not os.path.exists(f'{COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}/'):
        hf_hub_download(
            repo_id="JourneyDB/JourneyDB",
            repo_type="dataset",
            subfolder=f'data/{split}/imgs',
            filename=f'{idx:>03}.tgz',
            local_dir=COMPRESSED_DIR,
            local_dir_use_symlinks=False,
        )
    print(f"Downloaded idx: {idx}")

    print(f"Extracting idx: {idx}")
    if not os.path.exists(f'{COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}/'):
        subprocess.call(
            f'tar -xzf {COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}.tgz '
            f'-C {COMPRESSED_DIR}/data/{split}/imgs/',
            shell=True,
        )
    print(f"Extracted idx: {idx}")

    print(f"Removing idx: {idx}")
    if os.path.exists(f'{COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}.tgz'):
        os.remove(f'{COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}.tgz')
    print(f"Removed idx: {idx}")

    # add bicubic downsize
    downsize = transforms.Resize(
        max_image_size,
        antialias=True,
        interpolation=transforms.InterpolationMode.BICUBIC,
    )

    print(f"Downsizing idx: {idx}")
    os.makedirs(
        f'{RAW_DIR}/{split}/imgs/{idx:>03}/',
        exist_ok=True,
    )
    for f in iglob(f'{COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}/*'):
        save_path = f'{RAW_DIR}/{split}/imgs/{idx:>03}/{os.path.basename(f)}'

        if os.path.exists(save_path):
            continue

        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            try:
                img = Image.open(f)
                w, h = img.size
                if min(w, h) > max_image_size:
                    img = downsize(img)
                if min(w, h) < min_image_size:
                    print(
                        f'Skipping image with resolution ({h}, {w}) - '
                        f'Since at least one side has resolution below {min_image_size}'
                    )
                    continue

                img.save(save_path)
                os.remove(f)
            except (UnidentifiedImageError, OSError) as e:
                print(f"Error {e}, File: {f}")
    print(f'Downsized idx: {idx}')


if not os.path.exists(os.path.join(RAW_DIR, 'train/imgs')):

    valid_ids = list(np.arange(1))
    pool_args = [('train', i) for i in valid_ids] + [('valid', i) for i in valid_ids]
    max_image_size = 512
    min_image_size = 64

    with Pool(processes=4) as pool:
        pool.starmap(
            download_uncompress_resize,
            [(
                valid_ids,
                max_image_size,
                min_image_size,
                split,
                idx
            ) for split, idx in pool_args]
        )

### MDS形式に変換

In [ ]:
def convert_to_mds(
    images_dir: str,
    captions_jsonl: str,
    local_mds_dir: str,
):
    """
    JourneyDBをMDS形式に変換する
    MDS（Mosaic Data Storage）は、大規模データセット用の高速ストリーミングフォーマット

    Args:
        images_dir: 画像ディレクトリのパス
        captions_jsonl: キャプションのJSONLファイルのパス
        local_mds_dir: 出力MDSディレクトリのパス
    """

    columns = {
        'width': 'int32',
        'height': 'int32',
        'jpg': 'jpeg',
        'caption': 'str',
    }
    
    writer = MDSWriter(
        out=local_mds_dir,
        columns=columns,
        compression=None,
        size_limit=256 * (2**20),
        max_workers=64,
    )
    
    # Retrieving achieve indies, in case only a subset of the data is downloaded
    valid_archieve_idx = [
        os.path.basename(p) for p in glob(os.path.join(images_dir, '*'))
    ]
    
    metadata = list(open(captions_jsonl, 'r'))

    for f in tqdm(metadata):
        d = json.loads(f)
        cap, p = d['prompt'], d['img_path'].strip('./')
        
        if os.path.dirname(p) not in valid_archieve_idx:
            continue
            
        try:
            img = Image.open(os.path.join(images_dir, p))
            w, h = img.size
            mds_sample = {
                'jpg': img,
                'caption': cap,
                'width': w,
                'height': h,
            }
            writer.write(mds_sample)
        except Exception as e:
            print(
                "Something went wrong in reading caption, "
                f"skipping writing this sample in mds. Error: {e}"
            )

    writer.finish()

MDS_DIR = os.path.join(DATA_DIR, 'jdb', 'mds')
logger.info(f"{MDS_DIR=}")

if not os.path.exists(MDS_DIR):
    convert_to_mds(
        images_dir=os.path.join(RAW_DIR, 'train', 'imgs'),
        captions_jsonl=os.path.join(RAW_DIR, 'train', 'train_anno_realease_repath.jsonl'),
        local_mds_dir=os.path.join(MDS_DIR, 'train'),
    )

    convert_to_mds(
        images_dir=os.path.join(RAW_DIR, 'valid', 'imgs'),
        captions_jsonl=os.path.join(RAW_DIR, 'valid', 'valid_anno_repath.jsonl'),
        local_mds_dir=os.path.join(MDS_DIR, 'valid'),
    )

### 前処理

In [ ]:
class StreamingJdbDatasetForPreCompute(StreamingDataset):
    """Streaming dataset that resizes images to user-provided resolutions and tokenizes captions."""

    def __init__(
        self,
        streams: Sequence[Stream],
        transforms_list: List[Callable],
        batch_size: int,
        tokenizer_name: str,
        shuffle: bool = False,
        caption_key: str = 'caption',
    ):
        super().__init__(
            streams=streams,
            shuffle=shuffle,
            batch_size=batch_size,
        )

        self.transforms_list = transforms_list
        self.caption_key = caption_key
        self.tokenizer = UniversalTokenizer(tokenizer_name)
        print("Created tokenizer: ", tokenizer_name)
        assert self.transforms_list is not None, 'Must provide transforms to resize and center crop images'

    def __getitem__(self, index: int) -> Dict:
        sample = super().__getitem__(index)
        ret = {}

        out = self.tokenizer.tokenize(sample[self.caption_key])
        ret[self.caption_key] = out['input_ids'].clone().detach()
        if 'attention_mask' in out:
            ret[f'{self.caption_key}_attention_mask'] = out['attention_mask'].clone().detach()

        for i, tr in enumerate(self.transforms_list):
            img = sample['jpg']
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img = tr(img)
            ret[f'image_{i}'] = img

        ret['sample'] = sample
        return ret

In [ ]:
def build_streaming_jdb_precompute_dataloader(
    datadir: Union[List[str], str],
    batch_size: int,
    resize_sizes: Optional[List[int]] = None,
    drop_last: bool = False,
    shuffle: bool = True,
    caption_key: Optional[str] = None,
    tokenizer_name: Optional[str] = None,
    **dataloader_kwargs,
) -> DataLoader:
    """Builds a streaming mds dataloader returning multiple image sizes and text captions."""
    assert resize_sizes is not None, 'Must provide target resolution for image resizing'
    datadir = [datadir] if isinstance(datadir, str) else datadir
    streams = [Stream(remote=None, local=l) for l in datadir]

    transforms_list = []
    for resize in resize_sizes:
        transforms_list.append(
            transforms.Compose([
                transforms.Resize(
                    resize,
                    interpolation=transforms.InterpolationMode.BICUBIC,
                ),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        )

    dataset = StreamingJdbDatasetForPreCompute(
        streams=streams,
        shuffle=shuffle,
        transforms_list=transforms_list,
        batch_size=batch_size,
        caption_key=caption_key,
        tokenizer_name=tokenizer_name,
    )

    def custom_collate(list_of_dict: List[Dict]) -> Dict:
        out = {k: [] for k in list_of_dict[0].keys()}
        for d in list_of_dict:
            for k, v in d.items():
                out[k].append(v)
        return out

    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        drop_last=drop_last,
        collate_fn=custom_collate,
        **dataloader_kwargs,
    )

    return dataloader

In [ ]:
def precompute(
    datadir: str,
    savedir: str = "",
    image_resolutions: list = [256, 512],
    save_images: bool = False,
    model_dtype: str = "bfloat16",
    save_dtype: str = "float16",
    vae: str = "stabilityai/stable-diffusion-xl-base-1.0",
    text_encoder: str = "openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378",
    batch_size: int = 32,
    seed: int = 2024,
):
    """Precompute image and text latents and store them in MDS format.

    By default, we only save the image latents for 256x256 and 512x512 image
    resolutions (using center crop).

    Note that the image latents will be scaled by the vae_scaling_factor.
    """
    cap_key = 'caption'  # Hardcoding the image caption key to 'caption' in MDS dataset

    accelerator = Accelerator()
    device = accelerator.device
    device_idx = int(accelerator.process_index)

    # Set random seeds
    torch.manual_seed(device_idx + seed)
    torch.cuda.manual_seed(device_idx + seed)
    np.random.seed(device_idx + seed)

    dataloader = build_streaming_jdb_precompute_dataloader(
        datadir=[datadir],
        batch_size=batch_size,
        resize_sizes=image_resolutions,
        drop_last=False,
        shuffle=False,
        caption_key=cap_key,
        tokenizer_name=text_encoder,
        # prefetch_factor=2,
        # num_workers=2,
        # persistent_workers=True,
        pin_memory=True,
    )
    print(f'Device: {device_idx}, Dataloader sample count: {len(dataloader.dataset)}')

    # print(
    #     f"MP variable -> world size: {os.environ['WORLD_SIZE']}, "
    #     f"RANK: {os.environ['RANK']}, {device}"
    # )

    vae = AutoencoderKL.from_pretrained(
        vae,
        subfolder='vae',  # Change subfolder to appropriate one in hf_hub, if needed
        torch_dtype=DATA_TYPES[model_dtype],
    )
    print("Created VAE: ", vae)
    assert isinstance(vae, AutoencoderKL)

    text_encoder = UniversalTextEncoder(
        text_encoder,
        dtype=model_dtype,
        pretrained=True,
    )
    print("Created text encoder: ", text_encoder)

    vae = vae.to(device)
    text_encoder = text_encoder.to(device)

    columns = {
        cap_key: 'str',
        f'{cap_key}_latents': 'bytes',
        'latents_256': 'bytes',
        'latents_512': 'bytes',
    }
    if save_images:
        columns['jpg'] = 'jpeg'

    remote_upload = os.path.join(savedir, str(accelerator.process_index))

    writer = MDSWriter(
        out=remote_upload,
        columns=columns,
        compression=None,
        size_limit=256 * (2**20),
        max_workers=64,
    )

    for batch in tqdm(dataloader):
        image_256 = torch.stack(batch['image_0']).to(device)
        image_512 = torch.stack(batch['image_1']).to(device)
        captions = torch.stack(batch[cap_key]).to(device)

        with torch.no_grad():
            with torch.autocast(device_type='cuda', dtype=DATA_TYPES[model_dtype]):
                latent_dist_256 = vae.encode(image_256)
                assert isinstance(latent_dist_256, AutoencoderKLOutput)
                latents_256 = (
                    latent_dist_256['latent_dist'].sample().data * vae.config.scaling_factor
                ).to(DATA_TYPES[save_dtype])

                latent_dist_512 = vae.encode(image_512)
                assert isinstance(latent_dist_512, AutoencoderKLOutput)
                latents_512 = (
                    latent_dist_512['latent_dist'].sample().data * vae.config.scaling_factor
                ).to(DATA_TYPES[save_dtype])

                attention_mask = None

                if f'{cap_key}_attention_mask' in batch:
                    attention_mask = torch.stack(
                        batch[f'{cap_key}_attention_mask']
                    ).to(device)

                conditioning = text_encoder.encode(
                    captions.view(-1, captions.shape[-1]),
                    attention_mask=attention_mask,
                )[0].to(DATA_TYPES[save_dtype])

        try:
            if isinstance(latents_256, torch.Tensor) and isinstance(
                latents_512, torch.Tensor
            ):
                latents_256 = latents_256.detach().cpu().numpy()
                latents_512 = latents_512.detach().cpu().numpy()
            else:
                continue

            if isinstance(conditioning, torch.Tensor):
                conditioning = conditioning.detach().cpu().numpy()
            else:
                continue

            # Write the batch to the MDS file
            for i in range(latents_256.shape[0]):
                mds_sample = {
                    cap_key: batch['sample'][i][cap_key],
                    f'{cap_key}_latents': np.reshape(conditioning[i], -1).tobytes(),
                    'latents_256': latents_256[i].tobytes(),
                    'latents_512': latents_512[i].tobytes(),
                }
                if save_images:
                    mds_sample['jpg'] = batch['sample'][i]['jpg']
                writer.write(mds_sample)
        except RuntimeError:
            print('Runtime error CUDA, skipping this batch')

    writer.finish()

    # Wait for all processes to finish
    accelerator.wait_for_everyone()
    print(f'Process {accelerator.process_index} finished')
    time.sleep(10)

    # Merge the mds shards created by each device (only do on main process)
    if accelerator.is_main_process:
        shards_metadata = [
            os.path.join(savedir, str(i), 'index.json')
            for i in range(accelerator.num_processes)
        ]
        merge_index(shards_metadata, out=savedir, keep_local=True)

PRECOMPUTE_DIR = os.path.join(DATA_DIR, 'mds_latents_sdxl1_dfnclipH14',)
logger.info(f"{PRECOMPUTE_DIR=}")

if not os.path.exists(PRECOMPUTE_DIR):
    precompute(
        datadir=os.path.join(DATA_DIR, 'jdb', 'mds', 'train'),
        savedir=os.path.join(PRECOMPUTE_DIR, 'train'),
        image_resolutions=[256, 512],
        save_images=False,
        model_dtype="bfloat16",
        save_dtype="float16",
        vae="stabilityai/stable-diffusion-xl-base-1.0",
        text_encoder="openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378",
        batch_size=16,
        seed=2024,
    )

In [ ]:
assert os.path.exists(os.path.join(PRECOMPUTE_DIR, 'train', 'index.json'))

## 訓練

### ステップ１

In [ ]:
# Set up optimizer with special handling for MoE parameters

moe_params = [p[1] for p in model.dit.named_parameters() if 'moe' in p[0].lower()]
rest_params = [p[1] for p in model.dit.named_parameters() if 'moe' not in p[0].lower()]

if len(moe_params) > 0:
    print('Reducing learning rate of MoE parameters by 1/2')
    opt_dict = dict(cfg.optimizer)
    opt_name = opt_dict['_target_'].split('.')[-1]
    del opt_dict['_target_']
    optimizer = getattr(torch.optim, opt_name)(
        params=[{'params': rest_params}, {'params': moe_params, 'lr': cfg.optimizer.lr / 2}], **opt_dict)
else:
    optimizer = torch.optim.AdamW(
        params=model.dit.parameters(),
        lr=2.4e-4,
        weight_decay=0.1,
        eps=1.0e-8,
        betas=(0.9, 0.999)
    )

In [ ]:
# Convert ListConfig betas to native list to avoid ValueError when saving optimizer state
for p in optimizer.param_groups:
    p['betas'] = list(p['betas'])

In [ ]:
cap_seq_size, cap_emb_dim = text_encoder_embedding_format(
    'openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378'
)
logger.info(f"{cap_seq_size=}, {cap_emb_dim=}")

In [ ]:
class StreamingLatentsDataset(StreamingDataset):
    """Dataset class for loading precomputed latents from mds format.
    
    Args:
        streams: List of individual streams (in our case streams of individual datasets)
        shuffle: Whether to shuffle the dataset
        image_size: Size of images (256 or 512)
        cap_seq_size: Context length of text-encoder
        cap_emb_dim: Dimension of caption embeddings
        cap_drop_prob: Probability of using all zeros caption embedding (classifier-free guidance)
        batch_size: Batch size for streaming
    """

    def __init__(
        self,
        streams: Optional[List[Stream]] = None,
        shuffle: bool = False,
        image_size: Optional[int] = None,
        cap_seq_size: Optional[int] = None,
        cap_emb_dim: Optional[int] = None,
        cap_drop_prob: float = 0.0,
        batch_size: Optional[int] = None,
        **kwargs
    ) -> None:
        super().__init__(
            streams=streams,
            shuffle=shuffle,
            batch_size=batch_size,
        )

        self.image_size = image_size
        self.cap_seq_size = cap_seq_size
        self.cap_emb_dim = cap_emb_dim
        self.cap_drop_prob = cap_drop_prob

    def __getitem__(self, index: int) -> Dict[str, Union[torch.Tensor, str, float]]:
        sample = super().__getitem__(index)
        out = {}

        # Mask for zero'ed out captions in classifier-free guidance (cfg) training.
        # We replace caption embeddings with a zero vector in cfg guidance.
        out['drop_caption_mask'] = (
            0. if torch.rand(1) < self.cap_drop_prob else 1.
        )
        out['caption_latents'] = torch.from_numpy(
            np.frombuffer(sample['caption_latents'], dtype=np.float16)
            .copy()
        ).reshape(1, self.cap_seq_size, self.cap_emb_dim)

        if self.image_size == 256 and 'latents_256' in sample:
            out['image_latents'] = torch.from_numpy(
                np.frombuffer(sample['latents_256'], dtype=np.float16)
                .copy()
            ).reshape(-1, 32, 32)

        if self.image_size == 512 and 'latents_512' in sample:
            out['image_latents'] = torch.from_numpy(
                np.frombuffer(sample['latents_512'], dtype=np.float16)
                .copy()
            ).reshape(-1, 64, 64)

        # out['caption'] = sample['caption']
        return out

In [ ]:
def build_streaming_latents_dataloader(
    datadir: Union[str, List[str]],
    batch_size: int,
    image_size: int = 256,
    cap_seq_size: int = 77,
    cap_emb_dim: int = 1024,
    cap_drop_prob: float = 0.0,
    shuffle: bool = True,
    drop_last: bool = True,
    **dataloader_kwargs
) -> DataLoader:
    """Creates a DataLoader for streaming latents dataset."""
    if isinstance(datadir, str):
        datadir = [datadir]

    streams = [Stream(remote=None, local=d) for d in datadir]

    dataset = StreamingLatentsDataset(
        streams=streams,
        shuffle=shuffle,
        image_size=image_size,
        cap_seq_size=cap_seq_size,
        cap_emb_dim=cap_emb_dim,
        cap_drop_prob=cap_drop_prob,
        batch_size=batch_size,
    )

    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        sampler=None,
        drop_last=drop_last,
        **dataloader_kwargs,
    )

    return dataloader

In [ ]:
train_loader = build_streaming_latents_dataloader(
    datadir=['/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/'],
    image_size=256,
    batch_size=128 // dist.get_world_size(), # 2048
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    cap_drop_prob=0.1,
    drop_last=True,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

num_train_dataset_images = len(train_loader.dataset) * dist.get_world_size()
logger.info(f"{num_train_dataset_images=}")

In [ ]:
eval_loader = build_streaming_latents_dataloader(
    datadir='/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/',
    image_size=256,
    batch_size=1024 // dist.get_world_size(),
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    drop_last=False,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

num_eval_dataset_images = len(eval_loader.dataset) * dist.get_world_size()
logger.info(f"{num_eval_dataset_images=}")

In [ ]:
loggers = []
loggers.append(TensorboardLogger())

exp_name = 'MicroDiTXL_mask_75_res_256_pretrain'
wandb_logger = WandBLogger(
    init_kwargs={
        'name': f'{exp_name}',
        'project': 'microdit_training',  # insert wandb project name
        'group': f'{exp_name}'
    }
)
# loggers.append(wandb_logger)

In [ ]:
apply_low_precision_layernorm(
    model=model.dit,
    precision=Precision('amp_bf16'),
    optimizers=optimizer
)

algorithms = []
algorithms.append(
    GradientClipping(clipping_type='norm', clipping_threshold=0.25)
)

In [ ]:
callbacks = []
callbacks.append(SpeedMonitor(window_size=3))
callbacks.append(LRMonitor())
callbacks.append(RuntimeEstimator())
callbacks.append(OptimizerMonitor())
callbacks.append(
    LogDiffusionImages(
        prompts=[
            "a photograph of an astronaut riding a horse",
            "An astronaut riding a pig, highly realistic dslr photo, cinematic shot",
            "Panda mad scientist mixing sparkling chemicals, artstation",
            "a close-up of a fire spitting dragon, cinematic shot.",
            "A small cactus with a happy face in the Sahara desert",
            "Pirate ship trapped in a cosmic maelstrom nebula, rendered in cosmic beach whirlpool engine, volumetric lighting, spectacular, ambient lights, light pollution, cinematic atmosphere, art nouveau style, illustration art artwork by SenseiJaye, intricate detail.",
            "A dog that has been meditating all the time",
            "A Pikachu with an angry expression and red eyes, with lightning around it, hyper realistic style",
            "A worker that looks like a mixture of cow and horse is working hard to type code",
            "A capybara made of lego sitting in a realistic, natural field",
            "A grand piano with a white bench.",
            "In a fantastical setting, a highly detailed furry humanoid skunk with piercing eyes confidently poses in a medium shot, wearing an animal hide jacket. The artist has masterfully rendered the character in digital art, capturing the intricate details of fur and clothing texture.",
            "A illustration from a graphic novel. A bustling city street under the shine of a full moon. The sidewalks bustling with pedestrians enjoying the nightlife. At the corner stall, a young woman with fiery red hair, dressed in a signature velvet cloak, is haggling with the grumpy old vendor. the grumpy vendor, a tall, sophisticated man is wearing a sharp suit, sports a noteworthy moustache is animatedly conversing on his steampunk telephone.",
            "A fierce garden gnome warrior, clad in armor crafted from leaves and bark, brandishes a tiny sword and shield. He stands valiantly on a rock amidst a blooming garden, surrounded by colorful flowers and towering plants. A determined expression is painted on his face, ready to defend his garden kingdom.",
            "A giant cobra snake made from corn",
            "A green sign that says 'Very Deep Learning' and is at the edge of the Grand Canyon. Puffy white clouds are in the sky"
        ],
        guidance_scale=5,
        sampling_steps=30,
        seed=42
    )
)

callbacks.append(NaNCatcher())

In [ ]:
scheduler = CosineAnnealingWithWarmupScheduler(
    t_warmup='2500ba',
    alpha_f=0.33 # decay to 0.8e-4 after 256x256 masked pre-training
)

In [ ]:
# torch.compileにより15%程度高速化
compile = False

trainer = Trainer(
    model=model,
    train_dataloader=train_loader,
    eval_dataloader=eval_loader,
    optimizers=optimizer,
    schedulers=scheduler,
    loggers=loggers,
    algorithms=algorithms,
    callbacks=callbacks,
    device='gpu',
    # max_duration='250000ba',
    max_duration='1ba',
    eval_interval='2500ba' if compile else 0,
    save_interval='2500ba',
    save_num_checkpoints_to_keep=1,
    device_train_microbatch_size=256,
    run_name='microdit_experiment',
    seed=42,
    save_folder='./trained_models/microdit_experiment/',
    save_overwrite=True,
    autoresume=False,
    parallelism_config=None,
    precision='amp_bf16',  # if cfg.model['dtype'] == 'bfloat16' else 'amp_fp16',  # fp16 by default
    python_log_level='debug',
    compile_config={} if compile else None
)

In [ ]:
 # Ensure models are on correct device
device = next(model.dit.parameters()).device
model.vae.to(device)
model.text_encoder.to(device)

In [ ]:
logger.setLevel(logging.WARNING)  # to reduce streaming dataset info logs
trainer.fit()

### ステップ２

In [ ]:
# Set up optimizer with special handling for MoE parameters
moe_params = [p[1] for p in model.dit.named_parameters() if 'moe' in p[0].lower()]
rest_params = [p[1] for p in model.dit.named_parameters() if 'moe' not in p[0].lower()]

if len(moe_params) > 0:
    print('Reducing learning rate of MoE parameters by 1/2')
    opt_dict = dict(cfg.optimizer)
    opt_name = opt_dict['_target_'].split('.')[-1]
    del opt_dict['_target_']
    optimizer = getattr(torch.optim, opt_name)(
        params=[
            {'params': rest_params},
            {'params': moe_params, 'lr': 8e-5 / 2}], **opt_dict)
else:
    # optimizer = hydra.utils.instantiate(cfg.optimizer, params=model.dit.parameters())
    optimizer = torch.optim.AdamW(
        params=model.dit.parameters(),
        lr=8e-5,
        weight_decay=0.1,
        eps=1.0e-8,
        betas=(0.9, 0.999)
    )

In [ ]:
# Convert ListConfig betas to native list to avoid ValueError when saving optimizer state
for p in optimizer.param_groups:
    p['betas'] = list(p['betas'])

In [ ]:
# Set up data loaders

cap_seq_size, cap_emb_dim = text_encoder_embedding_format(
    'openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378'
)

train_loader = build_streaming_latents_dataloader(
    datadir=[
        '/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/'
    ],
    image_size=256,
    batch_size=128 // dist.get_world_size(), # 2048
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    cap_drop_prob=0.1,
    drop_last=True,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

eval_loader = build_streaming_latents_dataloader(
    datadir='/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/',
    image_size=256,
    batch_size=1024 // dist.get_world_size(),
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    drop_last=False,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)


In [ ]:
loggers = []
loggers.append(TensorboardLogger())

exp_name = 'MicroDiTXL_mask_0_res_256_finetune'
wandb_logger = WandBLogger(
    init_kwargs={
        'name': f'{exp_name}',
        'project': 'microdit_training',  # insert wandb project name
        'group': f'{exp_name}'
    }
)

In [ ]:
algorithms = []

apply_low_precision_layernorm(
    model=model.dit,
    precision=Precision('amp_bf16'),
    optimizers=optimizer
)

algorithms.append(
    GradientClipping(clipping_type='norm', clipping_threshold=0.25)
)

In [ ]:
callbacks = []
callbacks.append(SpeedMonitor(window_size=3))
callbacks.append(LRMonitor())
callbacks.append(RuntimeEstimator())
callbacks.append(OptimizerMonitor())
callbacks.append(
    LogDiffusionImages(
        prompts=[
            "a photograph of an astronaut riding a horse",
            "An astronaut riding a pig, highly realistic dslr photo, cinematic shot",
            "Panda mad scientist mixing sparkling chemicals, artstation",
            "a close-up of a fire spitting dragon, cinematic shot.",
            "A small cactus with a happy face in the Sahara desert",
            "Pirate ship trapped in a cosmic maelstrom nebula, rendered in cosmic beach whirlpool engine, volumetric lighting, spectacular, ambient lights, light pollution, cinematic atmosphere, art nouveau style, illustration art artwork by SenseiJaye, intricate detail.",
            "A dog that has been meditating all the time",
            "A Pikachu with an angry expression and red eyes, with lightning around it, hyper realistic style",
            "A worker that looks like a mixture of cow and horse is working hard to type code",
            "A capybara made of lego sitting in a realistic, natural field",
            "A grand piano with a white bench.",
            "In a fantastical setting, a highly detailed furry humanoid skunk with piercing eyes confidently poses in a medium shot, wearing an animal hide jacket. The artist has masterfully rendered the character in digital art, capturing the intricate details of fur and clothing texture.",
            "A illustration from a graphic novel. A bustling city street under the shine of a full moon. The sidewalks bustling with pedestrians enjoying the nightlife. At the corner stall, a young woman with fiery red hair, dressed in a signature velvet cloak, is haggling with the grumpy old vendor. the grumpy vendor, a tall, sophisticated man is wearing a sharp suit, sports a noteworthy moustache is animatedly conversing on his steampunk telephone.",
            "A fierce garden gnome warrior, clad in armor crafted from leaves and bark, brandishes a tiny sword and shield. He stands valiantly on a rock amidst a blooming garden, surrounded by colorful flowers and towering plants. A determined expression is painted on his face, ready to defend his garden kingdom.",
            "A giant cobra snake made from corn",
            "A green sign that says 'Very Deep Learning' and is at the edge of the Grand Canyon. Puffy white clouds are in the sky"
        ],
        guidance_scale=5,
        sampling_steps=30,
        seed=42
    )
)
callbacks.append(NaNCatcher())

In [ ]:
scheduler = ConstantScheduler(alpha=1.0)

In [ ]:
compile = False

fsdp_config = {
    'sharding_strategy': 'SHARD_GRAD_OP',
}

trainer = Trainer(
    model=model,
    train_dataloader=train_loader,
    eval_dataloader=eval_loader,
    optimizers=optimizer,
    schedulers=scheduler,
    loggers=loggers,
    algorithms=algorithms,
    callbacks=callbacks,
    device='gpu',
    # max_duration='280000ba',
    max_duration='1ba',
    eval_interval='2500ba',
    save_interval='2500ba',
    save_num_checkpoints_to_keep=1,
    device_train_microbatch_size=64,
    run_name='microdit_experiment',
    seed=18,
    save_folder='./trained_models/microdit_experiment/',
    # load_path='./trained_models/MicroDiTXL_mask_75_res_256_pretrain/latest-rank0.pt',
    # load_ignore_keys=[
    #     "state/optimizers/AdamW/param_groups/initial_lr",
    #     "state/optimizers/AdamW/param_groups/lr",
    #     "state/schedulers/LambdaLR/base_lrs",
    #     "state/schedulers/LambdaLR/_last_lr"
    # ],
    save_overwrite=True,
    autoresume=False,
    parallelism_config=None,
    precision='amp_bf16',  # if cfg.model['dtype'] == 'bfloat16' else 'amp_fp16',  # fp16 by default
    python_log_level='debug',
    compile_config={} if compile else None,
    # fsdp_config=fsdp_config
)

In [ ]:
device = next(model.dit.parameters()).device
model.vae.to(device)
model.text_encoder.to(device)

In [ ]:
trainer.fit()

### ステップ３

In [ ]:
# Set up optimizer with special handling for MoE parameters

moe_params = [p[1] for p in model.dit.named_parameters() if 'moe' in p[0].lower()]
rest_params = [p[1] for p in model.dit.named_parameters() if 'moe' not in p[0].lower()]

if len(moe_params) > 0:
    print('Reducing learning rate of MoE parameters by 1/2')
    opt_dict = dict(cfg.optimizer)
    opt_name = opt_dict['_target_'].split('.')[-1]
    del opt_dict['_target_']
    optimizer = getattr(torch.optim, opt_name)(
        params=[
            {'params': rest_params},
            {'params': moe_params, 'lr': 8e-5 / 2}], **opt_dict)
else:
    optimizer = torch.optim.AdamW(
        params=model.dit.parameters(),
        lr=8e-5,
        weight_decay=0.1,
        eps=1.0e-8,
        betas=(0.9, 0.999)
    )

In [ ]:
# Convert ListConfig betas to native list to avoid ValueError when saving optimizer state
for p in optimizer.param_groups:
    p['betas'] = list(p['betas'])

In [ ]:
# Set up data loaders
# cap_seq_size, cap_emb_dim = text_encoder_embedding_format(cfg.model.text_encoder_name)
cap_seq_size, cap_emb_dim = text_encoder_embedding_format(
    'openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378'
)

train_loader = build_streaming_latents_dataloader(
    datadir=[
        '/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/'
    ],
    image_size=256,
    batch_size=128 // dist.get_world_size(), # 2048
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    cap_drop_prob=0.0,
    drop_last=True,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

# print(f"Found {len(train_loader.dataset)*dist.get_world_size()} images in the training dataset")
# time.sleep(3)

eval_loader = build_streaming_latents_dataloader(
    datadir='/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/',
    image_size=256,
    batch_size=1024 // dist.get_world_size(),
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    drop_last=False,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

In [ ]:
loggers = []
loggers.append(TensorboardLogger())
exp_name = 'MicroDiTXL_mask_0_res_256_finetune_final'
wandb_logger = WandBLogger(
    init_kwargs={
        'name': f'{exp_name}',
        'project': 'microdit_training',  # insert wandb project name
        'group': f'{exp_name}'
    }
)
# loggers.append(wandb_logger)

In [ ]:
algorithms = []
apply_low_precision_layernorm(
    model=model.dit,
    precision=Precision('amp_bf16'),
    optimizers=optimizer
)
algorithms.append(
    GradientClipping(clipping_type='norm', clipping_threshold=0.25)
)

In [ ]:
callbacks = []
callbacks.append(SpeedMonitor(window_size=3))
callbacks.append(LRMonitor())
callbacks.append(RuntimeEstimator())
callbacks.append(OptimizerMonitor())
callbacks.append(
    LogDiffusionImages(
        prompts=[
            "a photograph of an astronaut riding a horse",
            "An astronaut riding a pig, highly realistic dslr photo, cinematic shot",
            "Panda mad scientist mixing sparkling chemicals, artstation",
            "a close-up of a fire spitting dragon, cinematic shot.",
            "A small cactus with a happy face in the Sahara desert",
            "Pirate ship trapped in a cosmic maelstrom nebula, rendered in cosmic beach whirlpool engine, volumetric lighting, spectacular, ambient lights, light pollution, cinematic atmosphere, art nouveau style, illustration art artwork by SenseiJaye, intricate detail.",
            "A dog that has been meditating all the time",
            "A Pikachu with an angry expression and red eyes, with lightning around it, hyper realistic style",
            "A worker that looks like a mixture of cow and horse is working hard to type code",
            "A capybara made of lego sitting in a realistic, natural field",
            "A grand piano with a white bench.",
            "In a fantastical setting, a highly detailed furry humanoid skunk with piercing eyes confidently poses in a medium shot, wearing an animal hide jacket. The artist has masterfully rendered the character in digital art, capturing the intricate details of fur and clothing texture.",
            "A illustration from a graphic novel. A bustling city street under the shine of a full moon. The sidewalks bustling with pedestrians enjoying the nightlife. At the corner stall, a young woman with fiery red hair, dressed in a signature velvet cloak, is haggling with the grumpy old vendor. the grumpy vendor, a tall, sophisticated man is wearing a sharp suit, sports a noteworthy moustache is animatedly conversing on his steampunk telephone.",
            "A fierce garden gnome warrior, clad in armor crafted from leaves and bark, brandishes a tiny sword and shield. He stands valiantly on a rock amidst a blooming garden, surrounded by colorful flowers and towering plants. A determined expression is painted on his face, ready to defend his garden kingdom.",
            "A giant cobra snake made from corn",
            "A green sign that says 'Very Deep Learning' and is at the edge of the Grand Canyon. Puffy white clouds are in the sky"
        ],
        guidance_scale=5,
        sampling_steps=30,
        seed=42
    )
)
callbacks.append(NaNCatcher())

In [ ]:
scheduler = ConstantScheduler(alpha=1.0)

In [ ]:
trainer = Trainer(
    model=model,
    train_dataloader=train_loader,
    eval_dataloader=eval_loader,
    optimizers=optimizer,
    schedulers=scheduler,
    loggers=loggers,
    algorithms=algorithms,
    callbacks=callbacks,
    device='gpu',
    # max_duration='50000ba',
    max_duration='1ba',
    eval_interval='2500ba',
    save_interval='2500ba',
    save_num_checkpoints_to_keep=1,
    device_train_microbatch_size=64,
    run_name='microdit_experiment',
    seed=18,
    save_folder='./trained_models/microdit_experiment/',
    load_path='./trained_models/MicroDiTXL_mask_0_res_256_finetune/latest-rank0.pt',
    load_weights_only=True,
    load_strict_model_weights=False,
    load_ignore_keys=["state/model/dit.pos_embed"],
    save_overwrite=True,
    autoresume=False,
    parallelism_config=None,
    precision='amp_bf16',  # if cfg.model['dtype'] == 'bfloat16' else 'amp_fp16',  # fp16 by default
    python_log_level='debug',
    compile_config={} if compile else None,
    # fsdp_config=fsdp_config
)

In [ ]:
device = next(model.dit.parameters()).device
model.vae.to(device)
model.text_encoder.to(device)

In [ ]:
trainer.fit()

### ステップ４

In [ ]:
# Set up optimizer with special handling for MoE parameters
moe_params = [p[1] for p in model.dit.named_parameters() if 'moe' in p[0].lower()]
rest_params = [p[1] for p in model.dit.named_parameters() if 'moe' not in p[0].lower()]
if len(moe_params) > 0:
    print('Reducing learning rate of MoE parameters by 1/2')
    opt_dict = dict(cfg.optimizer)
    opt_name = opt_dict['_target_'].split('.')[-1]
    del opt_dict['_target_']
    optimizer = getattr(torch.optim, opt_name)(
        params=[{'params': rest_params}, {'params': moe_params, 'lr': cfg.optimizer.lr / 2}], **opt_dict)
else:
    # optimizer = hydra.utils.instantiate(cfg.optimizer, params=model.dit.parameters())
    optimizer = torch.optim.AdamW(
        params=model.dit.parameters(),
        lr=8e-5,
        weight_decay=0.1,
        eps=1.0e-8,
        betas=(0.9, 0.999)
    )

# Convert ListConfig betas to native list to avoid ValueError when saving optimizer state
for p in optimizer.param_groups:
    p['betas'] = list(p['betas'])

In [ ]:
cap_seq_size, cap_emb_dim = text_encoder_embedding_format(
    'openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378'
)

train_loader = build_streaming_latents_dataloader(
    datadir=[
        '/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/'
    ],
    image_size=512,
    batch_size=128 // dist.get_world_size(), # 2048
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    cap_drop_prob=0.0,
    drop_last=True,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

eval_loader = build_streaming_latents_dataloader(
    datadir='/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/',
    image_size=512,
    batch_size=1024 // dist.get_world_size(),
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    drop_last=False,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

In [ ]:
loggers = []

loggers.append(TensorboardLogger())
exp_name = 'MicroDiTXL_mask_0_res_512_finetune_final'
wandb_logger = WandBLogger(
    init_kwargs={
        'name': f'{exp_name}',
        'project': 'microdit_training',  # insert wandb project name
        'group': f'{exp_name}'
    }
)
# loggers.append(wandb_logger)

In [ ]:
algorithms = []
apply_low_precision_layernorm(
    model=model.dit,
    precision=Precision('amp_bf16'),
    optimizers=optimizer
)
algorithms.append(
    GradientClipping(clipping_type='norm', clipping_threshold=0.25)
)

In [ ]:
callbacks = []
callbacks.append(SpeedMonitor(window_size=3))
callbacks.append(LRMonitor())
callbacks.append(RuntimeEstimator())
callbacks.append(OptimizerMonitor())
callbacks.append(
    LogDiffusionImages(
        prompts=[
            "a photograph of an astronaut riding a horse",
            "An astronaut riding a pig, highly realistic dslr photo, cinematic shot",
            "Panda mad scientist mixing sparkling chemicals, artstation",
            "a close-up of a fire spitting dragon, cinematic shot.",
            "A small cactus with a happy face in the Sahara desert",
            "Pirate ship trapped in a cosmic maelstrom nebula, rendered in cosmic beach whirlpool engine, volumetric lighting, spectacular, ambient lights, light pollution, cinematic atmosphere, art nouveau style, illustration art artwork by SenseiJaye, intricate detail.",
            "A dog that has been meditating all the time",
            "A Pikachu with an angry expression and red eyes, with lightning around it, hyper realistic style",
            "A worker that looks like a mixture of cow and horse is working hard to type code",
            "A capybara made of lego sitting in a realistic, natural field",
            "A grand piano with a white bench.",
            "In a fantastical setting, a highly detailed furry humanoid skunk with piercing eyes confidently poses in a medium shot, wearing an animal hide jacket. The artist has masterfully rendered the character in digital art, capturing the intricate details of fur and clothing texture.",
            "A illustration from a graphic novel. A bustling city street under the shine of a full moon. The sidewalks bustling with pedestrians enjoying the nightlife. At the corner stall, a young woman with fiery red hair, dressed in a signature velvet cloak, is haggling with the grumpy old vendor. the grumpy vendor, a tall, sophisticated man is wearing a sharp suit, sports a noteworthy moustache is animatedly conversing on his steampunk telephone.",
            "A fierce garden gnome warrior, clad in armor crafted from leaves and bark, brandishes a tiny sword and shield. He stands valiantly on a rock amidst a blooming garden, surrounded by colorful flowers and towering plants. A determined expression is painted on his face, ready to defend his garden kingdom.",
            "A giant cobra snake made from corn",
            "A green sign that says 'Very Deep Learning' and is at the edge of the Grand Canyon. Puffy white clouds are in the sky"
        ],
        guidance_scale=5,
        sampling_steps=30,
        seed=42
    )
)
callbacks.append(NaNCatcher())

In [ ]:
scheduler = ConstantScheduler(alpha=1.0)

In [ ]:
trainer = Trainer(
    model=model,
    train_dataloader=train_loader,
    eval_dataloader=eval_loader,
    optimizers=optimizer,
    schedulers=scheduler,
    loggers=loggers,
    algorithms=algorithms,
    callbacks=callbacks,
    device='gpu',
    # max_duration='55000ba',
    max_duration='1ba',
    eval_interval='500ba',
    save_interval='500ba',
    save_num_checkpoints_to_keep=1,
    device_train_microbatch_size=64,
    run_name='microdit_experiment',
    seed=18,
    save_folder='./trained_models/microdit_experiment/',
    load_path='./trained_models/MicroDiTXL_mask_0_res_256_finetune/latest-rank0.pt',
    load_weights_only=True,
    load_strict_model_weights=False,
    load_ignore_keys=["state/optimizers/AdamW/param_groups/initial_lr", "state/optimizers/AdamW/param_groups/lr", "state/schedulers/LambdaLR/base_lrs", "state/schedulers/LambdaLR/_last_lr"],
    save_overwrite=True,
    autoresume=False,
    parallelism_config=None,
    precision='amp_bf16',  # if cfg.model['dtype'] == 'bfloat16' else 'amp_fp16',  # fp16 by default
    python_log_level='debug',
    compile_config={} if compile else None,
    # fsdp_config=fsdp_config
)

In [ ]:
device = next(model.dit.parameters()).device
model.vae.to(device)
model.text_encoder.to(device)

In [ ]:
trainer.fit()